# Mona Fadaviardakani-96953781
# Domain-independent Text Segmentation

## Data-Loader: load data from different datasets and give them sample_type attributes

In [1]:
import os

import nltk
nltk.data.path.append("../nltk_data")

class LoadData:
    DIR="./data/"
    SUB_DIRS = {
        "fiction": "fiction/new/",
        "clinical": "clinical/",
        "biography": "biography/chapters/",
        "wikipedia": "wiki/"
    }

    def __init__(self):
        
        self.documents = []
        self.sent_tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

    ####### WIKIPEDIA DATA ##########################################################################################
    def load_wikipedia_sequence(self):
        SAMPLE_TYPE = 7     # 7th sample type
        doc = self.load_wikipedia()
        documents = []
        for wiki_page in doc:
            sequence = []
            for par in wiki_page:
                for i, line in enumerate(par):
                    sequence.append((line, int(i==0)))
            documents.append(sequence)
        return SAMPLE_TYPE, documents   # Each sample is a chapter

    
    def load_wikipedia(self):
        dirname = self.DIR + self.SUB_DIRS['wikipedia']
        
        # Use only .ref files
        files = sorted([dirname+fil for fil in os.listdir(dirname) if fil.endswith(".ref")])

        document = []          
        for fil in files:
            page = []
            with open(fil) as f:
                data = f.readlines()
            paragraph = []
            for line in data:
                line = line.strip()
                if line.startswith("======="):
                    if len(paragraph) > 0:
                        page.append(paragraph)
                    paragraph = []
                else:
                    paragraph.append(line)
            if len(paragraph) > 0:
                page.append(paragraph)
            document.append(page)

        print ("Total WIKIPEDIA(test) data: %d chapters, %d paragraphs and %d sentences" %(len(document), sum([len(chap) for chap in document]), sum([sum([len(par) for par in chap]) for chap in document])))
        return document

    ####### CLINICAL DATA ##########################################################################################
    def load_clinical_sequence(self):
        SAMPLE_TYPE = 4     # 4th sample type
        doc = self.load_clinical()
        documents = []
        for chapter in doc:
            sequence = []
            for par in chapter:
                for i, line in enumerate(par):
                    sequence.append((line, int(i==0)))
            documents.append(sequence)
        return SAMPLE_TYPE, documents   
    
    def load_clinical(self):
        dirname = self.DIR + self.SUB_DIRS['clinical']
        
        # Use only .ref files
        files = sorted([dirname+fil for fil in os.listdir(dirname) if fil.endswith(".ref")])

        document = []           
        for fil in files:
            chapter = []
            with open(fil) as f:
                data = f.readlines()
            paragraph = []
            for line in data:
                line = line.strip()
                if line.startswith("======="):
                    if len(paragraph) > 0:
                        chapter.append(paragraph)
                    paragraph = []
                else:
                    paragraph.append(line)
            if len(paragraph) > 0:
                chapter.append(paragraph)
            document.append(chapter)

        print ("Total CLINICAL data: %d chapters, %d paragraphs and %d sentences" %(len(document), sum([len(chap) for chap in document]), sum([sum([len(par) for par in chap]) for chap in document])))
        return document

    ####### BIOGRAPHY DATA ##########################################################################################
    def load_biography_sequence(self):
        SAMPLE_TYPE = 5     # 5th sample type
        doc = self.load_biography()
        sequence = []
        for chapter in doc:
            for par in chapter:
                for i, line in enumerate(par):
                    sequence.append((line, int(i==0)))
        return SAMPLE_TYPE, [sequence]   # Return a single sample with sequences of sentence

    
    def load_biography(self):
        dirname = self.DIR + self.SUB_DIRS['biography']
        
        # Use only .txt files
        files = sorted([dirname+fil for fil in os.listdir(dirname) if fil.endswith(".txt")])

        document = []           # Initialize document
        for fil in files:
            chapter = []
            with open(fil) as f:
                data = f.readlines()
            paragraph = []
            for line in data:
                line = line.strip().strip(".")      # Remove full-stop as well
                if line.startswith("======="):
                    if len(paragraph) > 0:
                        chapter.append(paragraph)
                    paragraph = []
                else:
                    if len(line) > 0:
                        paragraph.append(line)
            if len(paragraph) > 0:
                chapter.append(paragraph)
            document.append(chapter)

        print ("Total BIOGRAPHY data: %d chapters, %d paragraphs and %d sentences" %(len(document), sum([len(chap) for chap in document]), sum([sum([len(par) for par in chap]) for chap in document])))
        return document

    
    ####### FICTION DATA ##########################################################################################
    def load_fiction_sequence(self):
        SAMPLE_TYPE = 6     # 6th sample type
        doc_data = self.load_fiction()
        documents = []
        for book in doc_data:
            sequence = []
            for par in book:
                for i, line in enumerate(par):
                    sequence.append((line, int(i==0)))
            documents.append(sequence)
        return SAMPLE_TYPE, documents   # Return multiple sample with sequences of sentences. Each sample is a book

    
    def load_fiction(self, datatype='testing'):

        if datatype is 'testing':
            print (" Loading FICTION: testing data ")
        else:
            print ("Loading FICTION: development data")

        extension = '.dev' if datatype is 'development' else '.ref'
        dirname = self.DIR + self.SUB_DIRS['fiction']
        
        # Use appropriate file type for data
        files = sorted([dirname+fil for fil in os.listdir(dirname) if fil.endswith(extension)])

        document = []           # Initialize document
        for fil in files:
            book = []
            with open(fil) as f:
                data = f.readlines()
            paragraph = []
            for line in data:
                line = line.strip().strip(".")      # Remove full-stop as well
                if line.startswith("======="):
                    if len(paragraph) > 0:
                        book.append(paragraph[1:])  # Do not add the 1st line as it is the TITLE of the section
                    paragraph = []
                else:
                    if len(line) > 0:
                        # Each line does not have a single sentence. There are multiple sentences in a single line as well.
                        #paragraph.extend(self.sent_tokenizer.tokenize(line))    # Tokenizer splits a chunk of sentences into 1 or more sentences, so we need to extend the main list.
                        paragraph.append(line)
            if len(paragraph) > 0:
                book.append(paragraph[1:])      # Do not add the 1st line as it is the TITLE of the section
            document.append(book)

        print ("Total FICTION data: %d chapters, %d paragraphs and %d sentences" %(len(document), sum([len(chap) for chap in document]), sum([sum([len(par) for par in chap]) for chap in document])))
        return document

    

if __name__=="__main__":
    ld = LoadData()
    SAMPLE_TYPE_wiki,wiki_data=ld.load_wikipedia_sequence()
    SAMPLE_TYPE_cli, clinical_data = ld.load_clinical_sequence()
    SAMPLE_TYPE_bio, biography_data = ld.load_biography_sequence()
    SAMPLE_TYPE_fic, fiction_data = ld.load_fiction_sequence()


Total WIKIPEDIA(test) data: 300 chapters, 2265 paragraphs and 58578 sentences
Total CLINICAL data: 227 chapters, 909 paragraphs and 31868 sentences
Total BIOGRAPHY data: 11 chapters, 298 paragraphs and 2285 sentences
 Loading FICTION: testing data 
Total FICTION data: 82 chapters, 1245 paragraphs and 27551 sentences


# Example to see one samples of Clinical_data

In [3]:
print(len(fiction_data))
for _idx, sample in enumerate(fiction_data):
    print(_idx)
    print("''''''''''''''''''''''''")
    print(sample)
    break

82
0
''''''''''''''''''''''''
[('The balloon seemed scarcely to move, though it was slowly sinking toward the ocean of white clouds which hung between it and the earth. The two inmates of the car were insensible; their faces were bloodless, their cheeks sunken. They were both young and handsome. Harry Johnston, an American, was as dark and sallow as a Spaniard. Charles Thorndyke, an English gentleman, had yellow hair and mustache, blue eyes and a fine intellectual face. Both were tall, athletic in build and well-proportioned. Johnston was the first to come to consciousness as the balloon sank into less rarefied atmosphere. He opened his eyes dreamily and looked curiously at the white face of his friend in his lap. Then he shook him and tried to call his name, but his lips made no sound. Drawing himself up a little with a hand on the edge of the basket, he reached for a water-jug and sprinkled Thorndyke\'s face. In a moment he was rewarded by seeing the eyes of the latter slowly open. "

## Utility functions
* windiff_and_pk_metric_ONE_SEQUENCE : Remove the padded elements before calculating the windiff metric 
* unison_shuffled_copies= make shuffled copies of sequence
* compute_avg_seg_len= compute average based on the number of sentences
* precision, recall= compute these two metrics

In [151]:
import numpy as np
import pdb
from keras.models import model_from_json
from tabulate import tabulate
from keras import backend as K
import numpy as np
from keras.utils.np_utils import to_categorical
from theano.tensor import basic as tensor
import keras.backend as K


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], np.asarray(b)[p]


def round(arr):
    return np.round(np.array(arr, dtype=np.float64))

def compute_avg_seg_len(y_true):
    
    y_true = np.array(y_true)
    idx = np.where(y_true == 1)[0]
    seg_sizes, seg_count = [], idx.shape[0]
    for i in range(seg_count):
        if i == seg_count - 1:
            seg_sizes.append(y_true.shape[0] - idx[i])
        else:
            seg_sizes.append(idx[i+1] - idx[i])
    #print seg_sizes, np.mean(seg_sizes)
    if not len(seg_sizes):
        return 20.0
    return np.mean(seg_sizes)


def windiff_and_pk_metric_ONE_SEQUENCE(y_true, y_pred, window_size=-1, rounded=True, print_individual_stats=True):
    """ Make sure Y_pred is ROUNDED
    """
    
    # NOTE: WindowSize is the no of boundaries between the ith and jth sentence. i.e. no of sentences in between + 1

    assert y_true.shape[0] == y_pred.shape[0]

    average_seg_length = compute_avg_seg_len(y_true)
    if window_size == -1:
        window_size = int(average_seg_length * 0.5)   # WindowSize is equal to 1/2 of average window size of that document

    lenn = y_pred.shape[0]
    CUTOFF_VALUE=0.1
    if not rounded:

        y_pred = round(y_pred)

    # Convert all the values in single arrays for easy of comparisons and indexing
    t_cum, p_cum = np.cumsum(y_true, axis=0).reshape((1, lenn))[0], np.cumsum(y_pred, axis=0).reshape((1, lenn))[0]
    y_true_reshaped = y_true.reshape((1, lenn))[0]
    y_pred_reshaped = y_pred.reshape((1, lenn))[0]

    measurments, pk_differences, wd_differences = 0, 0, 0
    for i in range(0, lenn - window_size):
        j = i + window_size   # Their should be a total of "window_size" number of probes in between i and j

        # WinDiff
        
        ref_boundaries, hyp_boundaries = 0, 0

        ref_window, hyp_window = y_true[i: j+1], y_pred[i: j+1]
        for idx in range(0, window_size - 1):  # Iterate over all the elements of window

            if ref_window[idx] == 0 and ref_window[idx+1] == 1:   # Ref boundary exists
                ref_boundaries += 1

            if hyp_window[idx] == 0 and hyp_window[idx+1] == 1:   # Hyp boundary exists
                hyp_boundaries += 1

        if ref_boundaries != hyp_boundaries:
            wd_differences += 1

        # Pk
        
        agree_ref = t_cum[i] == t_cum[j]
        agree_hyp = p_cum[i] == p_cum[j]
        if agree_ref != agree_hyp:
            pk_differences += 1

        measurments += 1

    ans = {}
    ans['wd_r'] = (wd_differences*1.0)/(measurments)
    ans['wd_e'] = (wd_differences*1.0)/(measurments + 1)
    ans['pk'] = (pk_differences*1.0)/measurments

    if print_individual_stats:
        print (">> X:", y_true.shape, "| Avg_Seg_Length: %f | WinDiff_r: %f | WinDiff_e: %f | Pk: %f" %(average_seg_length, ans['wd_r'], ans['wd_e'], ans['pk']))

    return average_seg_length, ans

def precision(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall





## Preprocressing steps (normalize,Lemmatize, and remove stop words) and using word2vec to encode each word.

In [152]:
import numpy as np
import string

import nltk
nltk.data.path.append("../nltk_data")

from nltk.corpus import stopwords as s_words
from nltk.stem.wordnet import WordNetLemmatizer


class Dictionary:

    def __init__(self, EMBEDDING_DIM, word2vec_model):
        self.id2word_dic = {}
        self.word2id_dic = {}
        
        self.word2id_dic['<UNK>'] = 0
        self.id2word_dic[0] = '<UNK>'
        self.id_to_use_count = 1

        self.EMBEDDING_DIM = EMBEDDING_DIM
        self.word2vec = word2vec_model
        self.ignore_words = ['of', 'and', 'to', 'a']

        self.lemmatizer = WordNetLemmatizer()



    def normalize_word(self, word):
        # Do not lowercase, (`the` VS `The`)
        #word = word.lower()
        
        # Remove puct if needed
        if word in [w for w in string.punctuation]:
            return None

        # Remove stop words if needed
        if word in set(s_words.words('english')):
            return None

        return word


    def word2id(self, word):
        word = self.normalize_word(word)
        if word is None:
            return None

        try:
            return self.word2id_dic[word]
        except KeyError:
            if word not in self.word2vec:
                #if word not in self.ignore_words:       # Print words only if they do not belong to this list
                #    print "WORD2VEC:  `%s` not found" %(word)

                # Try to lemmatize the word and find its embedding
                # If not found then return UNK emb
                lm_word = self.lemmatizer.lemmatize(word)
                if lm_word not in self.word2vec:
                    return self.word2id_dic['<UNK>']
                #print "Lemmatized: %s ---> %s" %(word, lm_word)
                word = lm_word
                if word in self.word2id_dic:
                    return self.word2id_dic[word]

            self.word2id_dic[word] = self.id_to_use_count
            self.id2word_dic[self.id_to_use_count] = word
            self.id_to_use_count += 1
            return self.id_to_use_count - 1

    def id2word(self, wid):
        return self.id2word_dic[wid]

    def get_embedding_weights(self):
        embedding = np.zeros((len(self.word2id_dic) + 1, self.EMBEDDING_DIM))
        n = 0
        for k, v in self.word2id_dic.items():
            try:
                embedding[v] = self.word2vec[k]
            except Exception:
                print (Exception)
                n += 1
                pass
        print ("%d embedding missed"%n)
        #pdb.set_trace()
        return embedding

## Convert both sentence and the sequence of sentences to word2vec

In [154]:
import gensim

import nltk
nltk.data.path.append("../nltk_data")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as s_words

import pdb
import numpy as np
import string

import codecs

from keras.preprocessing.sequence import pad_sequences



def isINT(w):
    try:
        w = int(w)
    except ValueError:
        return 0
    return 1


class CustomSent2vec(object):
    """ Converts a sentence to a 2d representation of (AVERAGE_WORDS, EMBEDDING_DIM)
    """
    def __init__(self):
        print ("Loading word2vec model")
        #self.model = gensim.models.KeyedVectors.load_word2vec_format('/home/pinkesh.badjatiya/WORD_EMBEDDINGS/word2vec__GoogleNews_100Bwords__300Dvectors__3M_vocab.bin', binary=True)
        self.model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        #self.model = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print ("Model Loaded!")
        self.stopwords = set(s_words.words('english') + [w for w in string.punctuation])
        #self.vectorize = lambda x: self.model[x]
        self.dictionary = Dictionary(300, self.model)

        self.AVERAGE_WORDS = 20

    def get_dict_object(self):
        return self.dictionary

    def convert_sequence_sample_to_vec(self, sample, groundTruths):
        """ For type2 samples
        """
        # g_ths: Groundtruths
        sample_vec, g_ths = [], []
        for i, sentence in enumerate(sample):
            vec = []
            for w in word_tokenize(codecs.decode(sentence.encode(), "utf-8")):
                if not isINT(w):
                    word_id = self.dictionary.word2id(w)
                    if word_id is None: # Skip useless words, for words without any vector representation use the `<UNK>` word_id
                        continue
                    vec.append(word_id)

            if len(vec) > 0:
                sample_vec.append(np.asarray(vec))
                g_ths.append(groundTruths[i])

        # Check vstack() or hstack()
        X, Y = np.asarray(sample_vec), np.asarray(g_ths).reshape((len(g_ths), 1))
        X = pad_sequences(X, maxlen=self.AVERAGE_WORDS, padding="post", truncating="post", value=0.0, dtype=np.float32)
        return X, Y


   
    def convert_sample_to_vec(self, sample):
        """ For type1 samples
        """
        sample_vec = []
        for sentence in sample:
            vec = []
            for w in word_tokenize(codecs.decode(sentence.encode(), "utf-8")):
                if (w not in self.stopwords) and (not isINT(w)):
                    try:
                        vec.append(self.model[w])
                    except KeyError:
                        # Skip all the words whose vector representation is not present in the word2vec pre-trained model
                        continue
            if len(vec) > 0:
                sample_vec.append(np.mean(vec, axis=0))

        # Make the sentences in the sample equal to INPUT_VECTOR_LENGTH
        remove = len(sample_vec) - INPUT_VECTOR_LENGTH
        if remove > 0:
            print ("Vectorized sentence len not equal to min sentences in sample, INPUT_VECTOR_LENGTH")
            return None
        elif remove < 0:
            # Do not have sentences equal to INPUT_VECTOR_LENGTH
            #print ">>>>>>>>>>>> Found %d sentences instead of %d" %(len(sample_vec), INPUT_VECTOR_LENGTH)
            return None

        try:
            if len(sample_vec) != len(sample):
                raise Exception(">>>>>>>>> Found %d sentences in a sample instead of %d. (Skipped sentences while using word2vec)" %(len(sample_vec), len(sample)))
            
            
            temp = np.vstack(sample_vec)
            #temp = np.hstack(sample_vec)
        except Exception:
            print (">", Exception)
            
            return None
        return temp


if __name__=="__main__":
    word2vec = CustomSent2vec()
    

Loading word2vec model
Model Loaded!


## Simple Update and Progbar shows our progress in fitting data to neural net

In [43]:
import sys
import time
import datetime

def end():
    sys.stdout.write("\r\n")
    sys.stdout.flush()

SPACE_PAD = 30

def simple_update(prefix, count, total, speed=None, time_elapsed=None):
    # Should be called before running the timing code, as it is PRIOR printing on screen
    # Also, count is not 0 indexed. So it should be 1-indexed
    time_remaining = -1
    if speed:
        time_remaining = 1.0*(total - count + 1)*speed
    elif time_elapsed and count>1:  # since when count=1, we have not done anything.
        time_remaining = (1.0*(int(total) - count + 1)*time_elapsed)/(count - 1)

    time_remaining = str(datetime.timedelta(seconds=int(time_remaining))) if time_remaining != -1 else "---"

    sys.stdout.write('\r')
    if count == -1 or total == -1:
        sys.stdout.write("%s ..."%(prefix) + " "*SPACE_PAD)
    else:
        sys.stdout.write("%s: %s/%s ... (projected remaining time: %s)"%(prefix, count, total, time_remaining) + " "*SPACE_PAD)
    sys.stdout.flush()


def prog_bar(EVERYTHING_ZERO_INDEXED, total_samples, total_epochs, batch_size, epoch, batch_count, speed, data={}):
    assert EVERYTHING_ZERO_INDEXED == True  #   batch_count & epoch should be ZERO indexed
    total_batches = int((total_samples*1.0)/batch_size)
    print(total_batches)
    data_string = ", ".join([k+": %0.4f"%(v) for k, v in data.items()])

    # the exact output you're looking for:
    sys.stdout.write('\r')
    #sys.stdout.write("[%-60s] %d%%" % ('='*((60*(batch_count+1))/total_batches), (100*(batch_count+1))/total_batches))
    sys.stdout.flush()
    sys.stdout.write(", Epoc %d/%d, BatchSz: %d sents, Batch: %d/%d, Speed: %0.2f secs/batch, TimeRemain: %s || %s"% (epoch+1, total_epochs, batch_size, batch_count+1, total_batches, speed, str(datetime.timedelta(seconds=int(speed*(total_batches - (batch_count+1))))), data_string))
    sys.stdout.flush()


if __name__=="__main__":
    TOTAL_EPOCHS = 1
    TOTAL_SAMPLES = 100
    BATCH_SIZE = 1
    for e in range(TOTAL_EPOCHS):
        total_b = int((TOTAL_SAMPLES*1.0)/BATCH_SIZE)
        print(total_b)
        for batch_count in range(total_b):
        
            start = time.time()
            print(batch_count)
            
            # MAIN CODE
            time.sleep(1)
    
            speed = time.time() - start    # seconds per sample
            
            # print prog_bar
            prog_bar(True, TOTAL_SAMPLES, TOTAL_EPOCHS, BATCH_SIZE, e, batch_count, speed=speed)
        end()

100
0
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 1/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:39 || 1
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 2/100, Speed: 1.00 secs/batch, TimeRemain: 0:01:38 || 2
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 3/100, Speed: 1.00 secs/batch, TimeRemain: 0:01:37 || 3
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 4/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:36 || 4
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 5/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:35 || 5
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 6/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:34 || 6
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 7/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:33 || 7
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 8/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:32 || 8
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 9/100, Speed: 1.00 secs/batch, TimeRemain: 0:01:31 || 9
100
, Epoc 1/1, BatchSz: 1 sents, Batch: 10/100, Speed: 1.01 secs/batch, TimeRemain: 0:01:30 || 10
100
, Epoc 1/1, BatchS

# Handle the samples:
###  Returns X (Each row is a sample) , Y (A 1-D vector for ground truth), and also pad the sample input to the INPUT_VECTOR_LENGHT


In [36]:
from numpy import vstack
import numpy as np
from keras.preprocessing.sequence import pad_sequences

import time
#import load_data
#import progbar

AVERAGE_WORDS = 15
STATIC_PAD = 1

def get_input(sample_type, shuffle_documents, pad, trained_sent2vec_model=None):

    start = time.time()
    
    if sample_type == 4:
        # type4: Clinical sequence of a multiple samples
        # X.shape = (MULTIPLE_SAMPLES, TOTAL_SENTENCES)
        # Y.shape = (MULTIPLE_SAMPLES, TOTAL_SENTENCES, 1)
        #ld = load_data.LoadData()
        ld=LoadData()
        sample_type, samples = ld.load_clinical_sequence()
    elif sample_type == 5:
        # type5: Biography sequence of a single sample
        # X.shape = (1, TOTAL_SENTENCES)
        # Y.shape = (TOTAL_SENTENCES, 1)
        ld = LoadData()
        sample_type, samples = ld.load_biography_sequence()
    elif sample_type == 6:
        # type6: Fiction sequence of a multiple documents
        # X.shape = (NO_OF_BOOKS, TOTAL_SENTENCES)
        # Y.shape = (NO_OF_BOOKS, TOTAL_SENTENCES, 1)
        ld = LoadData()
        sample_type, samples = ld.load_fiction_sequence()
    elif sample_type == 7:
        # type7: Wiki sequence of a multiple sample
        # Data format is just like the clinical sequence as each line is a sentence
        # X.shape = (MULTIPLE_DOCUMENTS, TOTAL_SENTENCES)
        # Y.shape = (MULTIPLE_DOCUMENTS, TOTAL_SENTENCES, 1)
        ld = LoadData()
        sample_type, samples = ld.load_wikipedia_sequence()
    else:
        print ("NOTE: INVALID SAMPLE_TYPE!")
        return None

    print ("Samples Loading took", time.time() - start, "seconds")
    X=[]
    Y=[]

    model = trained_sent2vec_model
    if not trained_sent2vec_model:
        #model = TFIDF(samples)
        #model = MeanWord2vec()
        #model = TFIDFweightedMeanWord2vec(samples)
        model = CustomSent2vec()

    
    _total_samples = len(samples), time.time()
    #print len(samples)
    #pdb.set_trace()
    for _idx, sample in enumerate(samples):
        # Each sample is a document
        # Each sample is a list of tuples with each tuple as (sentence, groundTruth)
        sentences, groundTruths = zip(*sample)        # Unpack a sample

        # Traditional code
        if not _idx%50:
                time_elspsed=(time.time() - start)
                print("Converting doc to martices", _idx+1,"and the time passed is:",time_elspsed )

    
        if sample_type in ( 4, 5, 6, 7):
                sentences, groundTruths = model.convert_sequence_sample_to_vec(sentences, groundTruths)
        else:
                print ("Wrong Sample TYPE")

        if sentences is None:
                continue
        X.append(sentences)            # X[0].shape = matrix([[1,2,3,4.....]])
        Y.append(np.asarray(groundTruths))          # Y[0] = [1, 0, 0, ..... 0, 1, 0, 1....]
    #simple_update("Creating a standalone matrix for samples...", -1, -1)
    X, Y = np.asarray(X), np.asarray(Y)
    
    

    print ("Total samples: %d" %(len(X)))
    if shuffle_documents: # Shuffle the X's and Y's if required
        # Both of them have to be in unison
        X, Y = unison_shuffled_copies(X, Y)
        print ("SHUFFLE: Shuffled input document order! (X:",X.shape,", Y:",Y.shape,")")

    

    if pad:
        #### THIS PAD is messy!!!!
        ### Check once before padding
        if STATIC_PAD:
            max_len = AVERAGE_WORDS
        else:
            max_len = None  # Uses the max length of the sequences

        doc_lengths = [len(doc) for doc in X]
        print ("Padding sequences. Doc-lengths: Mean=%d, Std=%d" %(np.mean(doc_lengths), np.std(doc_lengths)))
        X = pad_sequences(X, padding="post", truncating="post", value=0.0, dtype=np.float32)
        Y = pad_sequences(Y, padding="post", truncating="post", value=0.0, dtype=np.float32)

        print ("Size of new X(after padding):", X.shape)

    return sample_type, X, Y, model
    #return sample_type, X, Y




if __name__=="__main__":
    get_input(sample_type=4, shuffle_documents=True, pad=False)
    

Total CLINICAL data: 227 chapters, 909 paragraphs and 31868 sentences
Samples Loading took 0.23184895515441895 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 67.43117094039917
Converting doc to martices 51 and the time passed is: 130.98965001106262
Converting doc to martices 101 and the time passed is: 182.75962090492249
Converting doc to martices 151 and the time passed is: 230.35364294052124
Converting doc to martices 201 and the time passed is: 269.76091599464417
Total samples: 227
SHUFFLE: Shuffled input document order! (X: (227,) , Y: (227,) )


## Implementing Attention Final Layer

In [44]:
from keras.layers.core import Layer
from keras import initializers, regularizers, constraints
from keras import backend as K

class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        
        eij= K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1)
        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number `epsilon` to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

In [45]:
from keras.layers.core import Layer
from keras import initializers, regularizers, constraints
from keras import backend as K


class AttentionWithContext(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        #uit = K.dot(x, self.W)
        uit= K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1)
        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        #ait = K.dot(uit, self.u)
        ait= K.squeeze(K.dot(uit, K.expand_dims(self.u)), axis=-1)
        if self.bias:
         a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number episilon to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

# Implementing CNN + Attn + BiLSTM (baseline Model)

In [46]:
from keras.layers import Concatenate

from keras.layers import Lambda
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, RepeatVector, Input, add, merge, Convolution1D, Convolution2D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.layers import concatenate
import numpy as np
import pdb
from nltk import tokenize
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold
import codecs
import gensim, sklearn
from string import punctuation
from collections import defaultdict
import sys
import math
import tensorflow as tf

from tabulate import tabulate

#from attention_lstm import AttentionLSTM
#from attention_lstm_without_weights import Attention
import time

trained_sample_handler = None
SAMPLE_TYPE_wiki, X_wiki, Y_wiki = None, None, None                 
#SAMPLE_TYPE_wikitest, X_wikitest, Y_wikitest = None, None, None    
SAMPLE_TYPE_cli, X_cli, Y_cli = None, None, None             
SAMPLE_TYPE_bio, X_bio, Y_bio = None, None, None             
SAMPLE_TYPE_fic, X_fic, Y_fic = None, None, None              

GLOVE_EMBEDDING_DIM = 300
SCALE_LOSS_FUN = True

# SEQUENCES_LENGTH_FOR_TRAINING = 40

ONE_SIDE_CONTEXT_SIZE = 10

LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN = False
SAVE_MODEL_AFTER_EACH_EPOCH = True
TRAINABLE_EMBEDDINGS = False


np.random.seed(2345)   # IMP seed
saved_model_epoch_done = None

def load_saved_model():
    global saved_model_epoch_done
    print ("====== Loading the saved model =======")
    filename = sys.argv[1]
    epoch_done = filename.split(".")[0].split("_")[-1]
    print (">>> Continuing from epoch %d <<<<" %(int(epoch_done)+1))
    saved_model_epoch_done = int(epoch_done)

    model = load_model(filename)
    return model
        

def reshape(x):
      return tf.reshape(x, shape=[-1,2400])
        
        
def baseline_perceptron(sequence_len):

    print ('Build MAIN model...')
    #pdb.set_trace()
    main_input = Input(shape=(sequence_len,), dtype='float32', name='main-input')

    encode_1 = Dense(300, name='dense-1')(main_input)
    drop_out_1 = Dropout(0.3)(encode_1)

    encode_2 = Dense(300, name='dense-2')(drop_out_1)
    drop_out_2 = Dropout(0.3)(encode_2)

    encode_3 = Dense(300, name='dense-3')(drop_out_2)
    drop_out_3 = Dropout(0.3)(encode_3)

    output = Dense(2, activation='sigmoid')(drop_out_3)
    

    model = Model(input=[main_input], output=output)
    #model.compile(loss=w_binary_crossentropy, optimizer='rmsprop', metrics=['accuracy', 'recall'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall])

    print (model.summary(line_length=150, positions=[.46, .65, .77, 1.]))
    return model



def lstm_model(sequences_length_for_training, embedding_dim, embedding_matrix, vocab_size):

    which_model = 2

    print ('Build MAIN model...')
    #pdb.set_trace()
    ngram_filters = [2, 3, 4, 5]
    conv_hidden_units = [200, 200, 200, 200]
    
    left_context= Input(shape=(ONE_SIDE_CONTEXT_SIZE+1, embedding_dim), dtype='float32', name='left-context')
    main_input = Input(shape=(1, embedding_dim), dtype='float32', name='main-input')
    right_context = Input(shape=(ONE_SIDE_CONTEXT_SIZE+1, embedding_dim), dtype='float32', name='right-context')

    context_embedder = TimeDistributed(Embedding(vocab_size + 1, GLOVE_EMBEDDING_DIM, weights=[embedding_matrix], input_length=embedding_dim, init='uniform'))
    main_input_embedder = TimeDistributed(Embedding(vocab_size + 1, GLOVE_EMBEDDING_DIM, weights=[embedding_matrix], input_length=embedding_dim, init='uniform'))
    embedded_input_left, embedded_input_main, embedded_input_right = context_embedder(left_context), main_input_embedder(main_input), context_embedder(right_context)

    convsL, convsM, convsR = [], [], []
    for n_gram, hidden_units in zip(ngram_filters, conv_hidden_units):
        conv_layer = Convolution1D(nb_filter=hidden_units,
                             filter_length=n_gram,
                             border_mode='same',
                             #border_mode='valid',
                             activation='tanh', name='Convolution-'+str(n_gram)+"gram")
        lef = TimeDistributed(conv_layer, name='TD-convolution-left-'+str(n_gram)+"gram")(embedded_input_left)
        mid = TimeDistributed(conv_layer, name='TD-convolution-mid-'+str(n_gram)+"gram")(embedded_input_main)
        rig = TimeDistributed(conv_layer, name='TD-convolution-right-'+str(n_gram)+"gram")(embedded_input_right)

        
        pool_L = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-left-'+str(n_gram)+"gram")(lef)
        pool_M = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-mid-'+str(n_gram)+"gram")(mid)
        pool_R = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-right-'+str(n_gram)+"gram")(rig)
        convsL.append(pool_L), convsM.append(pool_M), convsR.append(pool_R)
    
    convoluted_left=concatenate(convsL, axis=-1)  
    convoluted_mid=concatenate(convsM, axis=-1) 
    convoluted_right=concatenate(convsR, axis=-1) 
    CONV_DIM = sum(conv_hidden_units)
    #print(convoluted_left.shape)
    modeL=baseline_perceptron(sequence_length)
    #print(convoluted_mid.shape)

    ####convoluted_mid, convoluted_left, convoluted_right, CONV_DIM = main_input, left_context, right_context, 300
    flat_mid = Flatten()(convoluted_mid)
    encode_mid = Dense(300, name='dense-intermediate-mid-encoder')(flat_mid)

    context_encoder_intermediate1 = Bidirectional(LSTM(600, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=True, stateful=False), name='BiLSTM-context-encoder-intermediate1', merge_mode='concat')
    #context_encoder_intermediate2 = Bidirectional(LSTM(500, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.1, dropout_U=0.1, return_sequences=True, stateful=False), name='BiLSTM-context-encoder-intermediate2', merge_mode='concat')
    context_encoder = Bidirectional(LSTM(600, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=True, stateful=False), name='BiLSTM-context-encoder', merge_mode='concat')
    
    
    encode_left = AttentionWithContext(name='encode-left-attention')(context_encoder(context_encoder_intermediate1(convoluted_left)))
    encode_right = AttentionWithContext(name='encode-right-attention')(context_encoder(context_encoder_intermediate1(convoluted_right)))
    #encode_mid = AttentionWithContext(name='encode-mid-attention')(context_encoder1(convoluted_mid))
    ####encode_left = context_encoder(context_encoder_intermediate1(convoluted_left))
    ####encode_right = context_encoder(context_encoder_intermediate1(convoluted_right))
    print("encode-mid.shape",encode_mid.shape)
    encode_left_drop, encode_mid_drop, encode_right_drop = Dropout(0.3)(encode_left), Dropout(0.2)(encode_mid), Dropout(0.3)(encode_right)
    
    #encoded_info=concatenate([tf.reshape(encode_left_drop,[1,300]), encode_mid_drop, tf.reshape(encode_right_drop,[1,300])], axis=-1)
    #encode_left_drop=tf.reshape(encode_left_drop, shape=[-1, 300], name='encode_left_drop')
    #encode_right_drop=tf.reshape(encode_right_drop, shape=[-1, 300], name='encode_right_drop')
    #encode_left_drop = Lambda(reshape)(encode_left_drop)
    #encode_right_drop = Lambda(reshape)(encode_right_drop)
    print("sucessful")
    
    #XD = Lambda(concatenation,output_shape=(input_size,n+1))(encode_left_drop)
    encoded_info=Concatenate(axis=-1)([encode_left_drop,encode_right_drop])
    encoded_info = Lambda(reshape)(encoded_info)
    encoded_info=Concatenate(axis=-1)([encoded_info,encode_mid])
    #encoded_info = Merge(mode='concat', name='encode_info')([encode_left_drop, encode_mid_drop, encode_right_drop])

    decoded = Dense(500, name='decoded')(encoded_info)
    decoded_drop = Dropout(0.3, name='decoded_drop')(decoded)
    decoded1 = Dense(500, name='decoded1')(decoded_drop)
    decoded_drop1 = Dropout(0.3, name='decoded_drop1')(decoded1)
    output = Dense(1, activation='sigmoid')(decoded_drop1)
    model = Model(input=[left_context, main_input, right_context], output=output)
    #model = Model(input=[main_input], output=output)
    model.layers[1].trainable = TRAINABLE_EMBEDDINGS
    #model.compile(loss=w_binary_crossentropy, optimizer='rmsprop', metrics=['accuracy', 'recall'])
    model.compile(loss="binary_crossentropy", optimizer='adadelta', metrics=['accuracy', recall])


    print (model.summary(line_length=150, positions=[.46, .65, .77, 1.]))
    return model


def batch_gen_consecutive_context_segments_from_big_seq(typeOfBatch,X_with_doc, Y_with_doc, batch_size, one_side_context_size):
    # X should be (no_of_documents, total_sequences, bla, bla, bla ...)
    # Same should be Y
    X_Left_batch, Y_Left_batch, X_Right_batch, Y_Right_batch, X_Mid_batch, Y_Mid_batch = [], [], [], [], [], []
    for X, Y in zip(X_with_doc, Y_with_doc):   # iterate over document
        total_seq = X.shape[0]
        for i in range(total_seq):   # sample count
            # The neighbouring context do not contain the main sentence in their window
            #xL_i, xL_j, sent_i, xR_i, xR_j = i-one_side_context_size, i, i, i+1, i+one_side_context_size+1  # Example:: left: [0: 5] | Mid: [5: 6] | Right: [6: 11]
            #context_mat_size = one_side_context_size

            # The neighbouring context also contains the main sentence in the temporal data to derive some inference uing BLSTM
            xL_i, xL_j, sent_i, xR_i, xR_j = i-one_side_context_size, i+1, i, i, i+one_side_context_size+1  # Example:: left: [0: 6] | Mid: [5: 6] | Right: [5: 11]
            context_mat_size = one_side_context_size + 1

            if typeOfBatch == "train":    # Skip documents only when they are for training, for testing test on all the documents
                if total_seq < 2*one_side_context_size + 1:
                    print ("Too Small sequence: Found %d, required %d. DROPPING DOCUMENT" %(total_seq, 2*one_side_context_size+1))
                    break

            # Check if padding for the one_side_context_size required for both LEFT & RIGHT
            X_temp_Left, X_temp_Right, Y_temp_Left, Y_temp_Right = None, None, None, None
            if xL_i < 0:
                X_temp_Left, Y_temp_Left = np.array(X[0: xL_j]), np.array(Y[0: xL_j]) # Create resizable array not view
                
                shpX, shpY = list(X_temp_Left.shape), list(Y_temp_Left.shape)
                shpX[0] = shpY[0] = context_mat_size - xL_j
                X_temp_Left, Y_temp_Left = np.concatenate((np.zeros(shpX), X_temp_Left), axis=0), np.concatenate((np.zeros(shpY), Y_temp_Left), axis=0)
                
            else:
                X_temp_Left, Y_temp_Left = X[xL_i: xL_j], Y[xL_i: xL_j]

            if xR_j > total_seq:
                X_temp_Right, Y_temp_Right = np.array(X[xR_i: total_seq]), np.array(Y[xR_i: total_seq]) # Create resizable array not view
                
                shpX, shpY = list(X_temp_Right.shape), list(Y_temp_Right.shape)
                shpX[0] = shpY[0] = context_mat_size - (total_seq - xR_i)
                X_temp_Right, Y_temp_Right = np.concatenate((X_temp_Right, np.zeros(shpX)), axis=0), np.concatenate((Y_temp_Right, np.zeros(shpY)), axis=0)
                
            else:
                X_temp_Right, Y_temp_Right = X[xR_i: xR_j], Y[xR_i: xR_j]

            X_Left_batch.append(X_temp_Left), X_Right_batch.append(X_temp_Right)
            #Y_Left_batch.append(Y_Left_temp), Y_Right_batch.append(Y_Right_temp)       # No need for context groundtruths. May be we can test with that also later?
            X_Mid_batch.append(X[sent_i].reshape((1, -1))), Y_Mid_batch.append(Y[sent_i])  # X's should be TimeDistributed but not Y's
            if len(X_Left_batch) == batch_size:
                yield np.array(X_Left_batch), np.array(X_Mid_batch), np.asarray(X_Right_batch), np.array(Y_Mid_batch)
                X_Left_batch, Y_Left_batch, X_Right_batch, Y_Right_batch, X_Mid_batch, Y_Mid_batch = [], [], [], [], [], []
    if len(X_Left_batch):
        yield np.array(X_Left_batch), np.array(X_Mid_batch), np.asarray(X_Right_batch), np.array(Y_Mid_batch)


def batch_gen_sentences_without_context(X, Y, batch_size, fixed_size=False):
    
    total_seq = X.shape[0]
    for i in range(int(math.ceil(total_seq*1.0/batch_size))):   # sample count
        x_i, x_j = i*batch_size, (i+1)*batch_size
        X_temp, Y_temp = X[x_i: x_j], Y[x_i: x_j]
        count_sent = X_temp.shape[0]

        # Check if padding for the no of sentences required
        if fixed_size:
            if X_temp.shape[0] != batch_size:
                X_temp, Y_temp = np.array(X_temp), np.array(Y_temp) # Create resizable array not view
            shpX, shpY = list(X_temp.shape), list(Y_temp.shape)
            shpX[0], shpY[0] = batch_size, batch_size
            X_temp.resize(tuple(shpX)), Y_temp.resize(tuple(shpY))    # Resize to fit the batch size (ZEROES)

        yield np.array(X_temp), np.asarray(Y_temp)



def custom_fit(X_train, Y_train, X_test, Y_test, model, batch_size, epochs):

    # Print Train stats
    total_sentences, total_documents = 0, 0
    total_documents = X_train.shape[0]
    total_sentences = sum([doc.shape[0] for doc in X_train])
    print ("X-wiki TRAIN stats: Total %d sentences in %d documents" %(total_sentences, total_documents))

    class_weight = None
    if SCALE_LOSS_FUN:
        # Iterate as the no of sentences in each document is different
        # so np.unique() messes up.
        classes, counts = None, []
        for _temp_Yi in Y_train:
            classes, _temp_counts = np.unique(_temp_Yi, return_counts=True)
            counts.append(_temp_counts)
        counts = np.sum(counts, axis=0)
        class_weight = dict(zip(classes.tolist(), counts/float(sum(counts))))
        print (class_weight)

    train_avg_seg_len = np.mean([compute_avg_seg_len(Yi) for Yi in Y_train], axis=0)
    print (">> Train AVG_SEGMENT_LENGTH:", train_avg_seg_len)

    print ('Train...')
    start_epoch = 0
    if LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN:   # If we have saved model, then continue from the last epoch where we stopped
        start_epoch = saved_model_epoch_done  # The epoch count is zero indexed in TRAIN, while the count in saved file is 1 indexed

    for epoch in range(start_epoch, epochs):
        mean_tr_acc, mean_tr_loss, mean_tr_rec = [], [], []
        rLoss, rRecall, rAcc = 0,0,0 # Running parameters for printing while training
        for batch_count, (batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid) in enumerate(batch_gen_consecutive_context_segments_from_big_seq("train",X_train, Y_train, batch_size, ONE_SIDE_CONTEXT_SIZE)):
            #batch_Y_vec = to_categorical_MULTI_DIM(batch_Y, nb_classes=2)
                #batch_Y_mid = to_categorical(batch_Y_mid, num_classes=2) # Convert to output as 2 classes
                start = time.time()
                #print("batch_X_left",batch_X_left.shape)
                #print("batch_X_mid",batch_X_mid.shape)
                #print("batch_X_right", batch_X_right.shape)
                #print("batch_Y_mid", batch_Y_mid.shape)
                tr_loss, tr_acc, tr_rec = model.train_on_batch([batch_X_left, batch_X_mid, batch_X_right], batch_Y_mid)
                speed = time.time() - start
                mean_tr_acc.append(tr_acc)
                mean_tr_loss.append(tr_loss)
                mean_tr_rec.append(tr_rec)
                #rLoss, rRecall, rAcc = (rLoss*batch_count + tr_loss)/(batch_count + 1), (rRecall*batch_count + tr_rec)/(batch_count + 1), (rAcc*batch_count + tr_acc)/(batch_count + 1)
                #progbar.prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'rLoss': rLoss, 'rAcc': rAcc, 'rRec': rRecall })
                prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'Loss': tr_loss, 'Acc': tr_acc, 'Rec': tr_rec })
                # Print test results after every 100 batch trains
                
        if SAVE_MODEL_AFTER_EACH_EPOCH:
                model.save("model_trainable_%s_epoc_%d.h5" %(str(TRAINABLE_EMBEDDINGS), epoch+1))

        print (">> Epoch: %d/%d" %(epoch+1, epochs))
        print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
        print('recall training = {}'.format(np.mean(mean_tr_rec)))
        print('loss training = {}'.format(np.mean(mean_tr_loss)))

        #testing_on_data("Wikipedia(DEVELOPMENT)", X_test, Y_test, model, batch_size, summary_only=True)
        #save_predictions("Clinical", X_cli, Y_cli, model, batch_size, summary_only=True)
        #save_predictions("Biography", X_bio, Y_bio, model, batch_size, summary_only=True)
        #save_predictions("Fiction", X_fic, Y_fic, model, batch_size, summary_only=True)
        

        print('___________________________________')

   # Testing
    print ("####################################################################")
    print (">> (TEST) >> Testing, X:", X_test.shape, "Y:", Y_test.shape)
    mean_te_acc, mean_te_loss, mean_te_rec = [], [], []
    for i in range(X_test.shape[0]):
        X, Y = X_test[i], Y_test[i]
        X, Y = X.reshape((1,) + X.shape), Y.reshape((1,) + Y.shape)   # Convert to format of 1 document
        for batch_count, (batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid) in enumerate(batch_gen_consecutive_context_segments_from_big_seq("test",X, Y, batch_size, ONE_SIDE_CONTEXT_SIZE)):
            
            te_loss, te_acc, te_rec = model.test_on_batch([batch_X_left, batch_X_mid, batch_X_right], batch_Y_mid)
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
            mean_te_rec.append(te_rec)

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('recall testing = {}'.format(np.mean(mean_te_rec)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
        

def save_predictions(type_of_data, X_test, Y_test, model, batch_size, summary_only=False):
    # Predicting
    print ("====================== %s ======================" %(type_of_data))
    print ("GET PREDICTIONS... (SEPARATELY FOR EACH DOCUMENT)")
    data = {
        'wd_r': [],
        'wd_e': [],
        'pk': []
    }
    doc_idx = 18
    avg_segment_lengths_across_test_data = [] # Average segment length across the documents
    predictions_return = []
    
    for Xi_test, Yi_test in zip(X_test, Y_test):
        pred_per_doc = []
        Xi_test, Yi_test = Xi_test.reshape((1,) + Xi_test.shape), Yi_test.reshape((1,) + Yi_test.shape)   # Convert to format of 1 document
        for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq("test",Xi_test, Yi_test, batch_size, ONE_SIDE_CONTEXT_SIZE):
            batch_y_pred = model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right])
            pred_per_doc.append(batch_y_pred)


        #rounded = np.round(pred_per_doc)
    pred_per_doc = np.concatenate(pred_per_doc, axis=0)
        
    actual_avg_seg_length, result = windiff_and_pk_metric_ONE_SEQUENCE(Yi_test[0], pred_per_doc, window_size=-1, rounded=False, print_individual_stats=not summary_only)
    avg_segment_lengths_across_test_data.append(actual_avg_seg_length)
    data['pk'].append(result['pk'])
    data['wd_r'].append(result['wd_r'])
    data['wd_e'].append(result['wd_e'])

    print (">> Summary (%s):" %(type_of_data))
    print ("AVG segment length in test data: %f, std: %f" % (np.mean(avg_segment_lengths_across_test_data), np.std(avg_segment_lengths_across_test_data)))
    print ("WinDiff metric (Windiff_r):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_r']), np.std(data['wd_r']), np.min(data['wd_r']), np.max(data['wd_r'])))
    print ("WinDiff metric (Windiff_e):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_e']), np.std(data['wd_e']), np.min(data['wd_e']), np.max(data['wd_e'])))
    print ("Pk metric:: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['pk']), np.std(data['pk']), np.min(data['pk']), np.max(data['pk'])))
    print('___________________________________')



def testing_on_data(type_of_data, X_test, Y_test, model, batch_size, summary_only=False, visualize=False):
    # Predicting
    print ("====================== %s ======================" %(type_of_data))
    print ("Predicting... (SEPARATELY FOR EACH DOCUMENT)")
    data = {
        'wd_r': [],
        'wd_e': [],
        'pk': []
    }
    avg_segment_lengths_across_test_data = [] # Average segment length across the documents
    for Xi_test, Yi_test in zip(X_test, Y_test):
        pred_per_doc = []
        #Xi_test, Yi_test = Xi_test.reshape((1,) + Xi_test.shape), Yi_test.reshape((1,) + Yi_test.shape)   # Convert to format of 1 document
        for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq(Xi_test, Yi_test, batch_size, ONE_SIDE_CONTEXT_SIZE):
            batch_y_pred = model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right])
            pred_per_doc.append(batch_y_pred)

        if not len(pred_per_doc): # batch generator might drop a few documents
            continue

        #rounded = np.round(pred_per_doc)
        pred_per_doc = np.concatenate(pred_per_doc, axis=0)
        actual_avg_seg_length, result = windiff_and_pk_metric_ONE_SEQUENCE(Yi_test[0], pred_per_doc, window_size=-1, rounded=False, print_individual_stats=not summary_only)
        avg_segment_lengths_across_test_data.append(actual_avg_seg_length)
        data['pk'].append(result['pk'])
        data['wd_r'].append(result['wd_r'])
        data['wd_e'].append(result['wd_e'])

    print (">> Summary (%s):" %(type_of_data))
    print ("AVG segment length in test data: %f" % (np.mean(avg_segment_lengths_across_test_data)))
    print ("WinDiff metric (Windiff_r):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_r']), np.std(data['wd_r']), np.min(data['wd_r']), np.max(data['wd_r'])))
    print ("WinDiff metric (Windiff_e):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_e']), np.std(data['wd_e']), np.min(data['wd_e']), np.max(data['wd_e'])))
    print ("Pk metric:: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['pk']), np.std(data['pk']), np.min(data['pk']), np.max(data['pk'])))
    print('___________________________________')
    
def train(X_train, Y_train, model, batch_size, epochs=10):
    
    total_sentences, total_documents = 0, 0
    total_documents = X_train.shape[0]
    total_sentences = sum([doc.shape[0] for doc in X_train])
    #print ( "X-wiki TRAIN stats: Total %d sentences in %d documents" %(total_sentences, total_documents))

    class_weight = None
    if SCALE_LOSS_FUN:
        # Iterate as the no of sentences in each document is different
        # so np.unique() messes up.
        classes, counts = None, []
        for _temp_Yi in Y_train:
            classes, _temp_counts = np.unique(_temp_Yi, return_counts=True)
            counts.append(_temp_counts)
        counts = np.sum(counts, axis=0)
        class_weight = dict(zip(classes.tolist(), counts/float(sum(counts))))
        print (class_weight)

    train_avg_seg_len = np.mean([compute_avg_seg_len(Yi) for Yi in Y_train], axis=0)
    #print (">> Train AVG_SEGMENT_LENGTH:", train_avg_seg_len)
   
    #print ('Train...')
    start_epoch = 0
    outputs=np.zeros(6)
    if LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN:   # If we have saved model, then continue from the last epoch where we stopped
        start_epoch = saved_model_epoch_done  # The epoch count is zero indexed in TRAIN, while the count in saved file is 1 indexed

    print_iter_count = 0
    for epoch in range(start_epoch, epochs):
        mean_tr_acc, mean_tr_loss, mean_tr_rec = [], [], []
        batch_count = 0
        rLoss, rRecall, rAcc = 0,0,0 # Running parameters for printing while training
        for i in range(total_documents):
            X, Y = X_train[i], Y_train[i]
            for (batch_X, batch_Y) in batch_gen_sentences_without_context(X, Y, batch_size, fixed_size=False):
            
                batch_Y = to_categorical(batch_Y, num_classes=2) # Convert to output as 2 classes

                start = time.time()
                #print("batch_X.shape",batch_X.shape)
                #print("batch_Y.shape",batch_Y.shape)
                tr_loss, tr_acc, tr_rec = model.train_on_batch([batch_X], batch_Y)
                speed = time.time() - start

                mean_tr_acc.append(tr_acc)
                mean_tr_loss.append(tr_loss)
                mean_tr_rec.append(tr_rec)
                #rLoss, rRecall, rAcc = (rLoss*batch_count + tr_loss)/(batch_count + 1), (rRecall*batch_count + tr_rec)/(batch_count + 1), (rAcc*batch_count + tr_acc)/(batch_count + 1)
                #progbar.prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'rLoss': rLoss, 'rAcc': rAcc, 'rRec': rRecall })
                prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'Loss': tr_loss, 'Acc': tr_acc, 'Rec': tr_rec })
                batch_count += 1

       
        
        print (">> Epoch: %d/%d" %(epoch+1, epochs))
        print('accuracy = {}'.format(np.mean(mean_tr_acc)))
        #print('recall training = {}'.format(np.mean(mean_tr_rec)))
        print('loss training = {}'.format(np.mean(mean_tr_loss)))

        #testing_on_data("Wikipedia(DEVELOPMENT)", X_test, Y_test, model, batch_size, summary_only=True)
        testing_on_data("Clinical", X_cli, Y_cli, model, batch_size, summary_only=True)
        testing_on_data("Biography", X_bio, Y_bio, model, batch_size)
        testing_on_data("Fiction", X_fic, Y_fic, model, batch_size, summary_only=True)
        #testing_on_data("Wikipedia(BENCHMARK)", X_wikitest, Y_wikitest, model, batch_size, summary_only=True)
        return outputs
    

def split_data(X, Y, train_split):
    samples = X.shape[0]
    train_samples = int(X.shape[0]*train_split)
    X_train, Y_train = X[:train_samples+1,], Y[:train_samples+1,]
    X_test, Y_test = X[train_samples+1:,], Y[train_samples+1:,]
    print ("X_train:", X_train.shape, " | Y_train:", Y_train.shape)
    print ("X_test:", X_test.shape, " | Y_test:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test


def train_model(X, Y, model, train_split=0.8, epochs=5, batch_size=32):
    global X_wiki, Y_wiki, X_cli, Y_cli, X_bio, Y_bio, X_fic, Y_fic

    which_model = 2

    # This is only for training! (If train_split =1 then only TEST)
    X_train, Y_train, X_test, Y_test = split_data(X, Y, train_split=train_split)

    if which_model == 2:
        #custom_fit(X_train, Y_train, X_test, Y_test, model=model, batch_size=batch_size, epochs=epochs)
        custom_fit(X_train, Y_train, X_test, Y_test, model=model, batch_size=batch_size, epochs=epochs)
        
        print (">>>>>> Final Testing <<<<<<")
        print (">> ATTENTION weights:")
        print ("################# Left - attention - weights ")
        attn_weights_left = [model.get_layer("encode-left-attention").get_weights(), model.get_layer("encode-left-attention").get_weights()]
        print (attn_weights_left[0])
        print (attn_weights_left[1])
          
        attn_weights_right = [model.get_layer("encode-right-attention").get_weights(), model.get_layer("encode-right-attention").get_weights()]
        print ("################# RIght - attention - weights ")
        print (attn_weights_right[0])
        print (attn_weights_right[1])

       
    
    
if __name__ == "__main__":

    # Print parameters
    print ("=== SCALE_LOSS_FUN: %d, ONE_SIDE_CONTEXT_SIZE: %d ===" % (int(SCALE_LOSS_FUN), ONE_SIDE_CONTEXT_SIZE))
    
    print ("NOTE: Make sure you have MIN_SENTENCES_IN_DOCUMENT >= 2*context_size + 1")

    # For which_model == 2
    SAMPLE_TYPE_wiki, X_wiki, Y_wiki, trained_sample_handler = get_input(sample_type=7, shuffle_documents=True, pad=False)
    modal=baseline_perceptron(20)
    NO_OF_SAMPLES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = X_wiki.shape[0], -1, X_wiki[0].shape[1]          #MAX_SEQUENCE_LENGTH is is already padded
    
    # For which_model == 2
    # Biography data for training
    SAMPLE_TYPE_bio, X_bio, Y_bio, trained_sample_handler = get_input(sample_type=5, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    NO_OF_SAMPLES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = X_bio.shape[0], -1, X_bio[0].shape[1]          #MAX_SEQUENCE_LENGTH is is already padded
    
    # Only for testing
    SAMPLE_TYPE_cli, X_cli, Y_cli, trained_sample_handler = get_input(sample_type=4, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    SAMPLE_TYPE_fic, X_fic, Y_fic, trained_sample_handler = get_input(sample_type=6, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    #SAMPLE_TYPE_wikitest, X_wikitest, Y_wikitest, trained_sample_handler = get_input(sample_type=7, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    
    #print "Check data type"

    sequence_length = X_wiki[0].shape[-1]
    dictionary_object = trained_sample_handler.dictionary
    embedding_W = dictionary_object.get_embedding_weights()

    # COmment temporarily for context variation experiemnt
    if LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN:
        model = load_saved_model()
    else:
        #model = baseline_perceptron(sequence_length)
        model = lstm_model(-1, EMBEDDING_DIM, embedding_W, len(dictionary_object.word2id_dic))

    train_model(X_wiki, Y_wiki, model, train_split=0.8, epochs=5, batch_size=64)
    ##train_LSTM(X_bio, Y_bio, model, embedding_W, train_split=0.7, epochs=1, batch_size=32)





=== SCALE_LOSS_FUN: 1, ONE_SIDE_CONTEXT_SIZE: 10 ===
NOTE: Make sure you have MIN_SENTENCES_IN_DOCUMENT >= 2*context_size + 1
Total WIKIPEDIA(test) data: 300 chapters, 2265 paragraphs and 58578 sentences
Samples Loading took 0.37503576278686523 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 85.37636399269104
Converting doc to martices 51 and the time passed is: 158.60315084457397
Converting doc to martices 101 and the time passed is: 246.74908781051636
Converting doc to martices 151 and the time passed is: 338.58198595046997
Converting doc to martices 201 and the time passed is: 420.204069852829
Converting doc to martices 251 and the time passed is: 470.1119339466095
Total samples: 300
SHUFFLE: Shuffled input document order! (X: (300,) , Y: (300,) )
Total BIOGRAPHY data: 11 chapters, 298 paragraphs and 2285 sentences
Samples Loading took 0.034848928451538086 seconds
Converting doc to martices 1 and the time passed is: 0.046391010284423

/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(80472, 300, weights=[array([[ ..., input_length=20, embeddings_initializer="uniform")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(80472, 300, weights=[array([[ ..., input_length=20, embeddings_initializer="uniform")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-2gram", filters=200, kernel_size=2, padding="same")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-3gram", filters=200, kernel_size=3, padding="same")`
/Users/monaafadavi/anaconda3/l

encode-mid.shape (?, 300)
sucessful
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                         Output Shape                Param #           Connected to                       
left-context (InputLayer)                                            (None, 11, 20)              0                                                    
______________________________________________________________________________________________________________________________________________________
right-context (InputLayer)                                           (None, 11, 20)              0                                                    
______________________________________________________________________________________________________________________________________________________
time_distributed_3 (TimeDistributed)                      

/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


{0: 0.9611758417958312, 1: 0.038824158204168895}
>> Train AVG_SEGMENT_LENGTH: 25.937006902597574
Train...
730
, Epoc 1/5, BatchSz: 64 sents, Batch: 1/730, Speed: 21.70 secs/batch, TimeRemain: 4:23:37 || Loss: 0.6637, Acc: 0.7031, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 2/730, Speed: 5.15 secs/batch, TimeRemain: 1:02:28 || Loss: 0.1812, Acc: 0.9531, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 3/730, Speed: 5.35 secs/batch, TimeRemain: 1:04:47 || Loss: 0.1394, Acc: 0.9688, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 4/730, Speed: 5.01 secs/batch, TimeRemain: 1:00:34 || Loss: 0.1200, Acc: 0.9688, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 5/730, Speed: 5.86 secs/batch, TimeRemain: 1:10:47 || Loss: 0.2061, Acc: 0.9531, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 6/730, Speed: 5.66 secs/batch, TimeRemain: 1:08:16 || Loss: 0.1885, Acc: 0.9531, Rec: 0.0000730
, Epoc 1/5, BatchSz: 64 sents, Batch: 7/730, Speed: 6.39 secs/batch, TimeRemain: 1:17

...

, Epoc 5/5, BatchSz: 64 sents, Batch: 312/730, Speed: 5.22 secs/batch, TimeRemain: 0:36:20 || Loss: 0.0069, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 313/730, Speed: 5.13 secs/batch, TimeRemain: 0:35:38 || Loss: 0.0171, Acc: 0.9844, Rec: 0.8000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 314/730, Speed: 5.22 secs/batch, TimeRemain: 0:36:10 || Loss: 0.0005, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 315/730, Speed: 5.31 secs/batch, TimeRemain: 0:36:42 || Loss: 0.0037, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 316/730, Speed: 5.30 secs/batch, TimeRemain: 0:36:35 || Loss: 0.0071, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 317/730, Speed: 5.16 secs/batch, TimeRemain: 0:35:32 || Loss: 0.0641, Acc: 0.9688, Rec: 0.5000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 318/730, Speed: 5.13 secs/batch, TimeRemain: 0:35:12 || Loss: 0.0031, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 319/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 432/730, Speed: 5.11 secs/batch, TimeRemain: 0:25:22 || Loss: 0.0450, Acc: 0.9844, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 433/730, Speed: 5.28 secs/batch, TimeRemain: 0:26:09 || Loss: 0.0089, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 434/730, Speed: 5.12 secs/batch, TimeRemain: 0:25:14 || Loss: 0.0093, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 435/730, Speed: 5.17 secs/batch, TimeRemain: 0:25:24 || Loss: 0.0015, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 436/730, Speed: 5.14 secs/batch, TimeRemain: 0:25:10 || Loss: 0.0006, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 437/730, Speed: 5.25 secs/batch, TimeRemain: 0:25:38 || Loss: 0.0014, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 438/730, Speed: 5.14 secs/batch, TimeRemain: 0:25:01 || Loss: 0.0006, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 439/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 552/730, Speed: 5.12 secs/batch, TimeRemain: 0:15:11 || Loss: 0.0023, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 553/730, Speed: 5.13 secs/batch, TimeRemain: 0:15:07 || Loss: 0.0006, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 554/730, Speed: 5.10 secs/batch, TimeRemain: 0:14:58 || Loss: 0.0397, Acc: 0.9844, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 555/730, Speed: 5.21 secs/batch, TimeRemain: 0:15:11 || Loss: 0.1278, Acc: 0.9844, Rec: 0.5000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 556/730, Speed: 5.21 secs/batch, TimeRemain: 0:15:05 || Loss: 0.0323, Acc: 0.9688, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 557/730, Speed: 5.05 secs/batch, TimeRemain: 0:14:33 || Loss: 0.1842, Acc: 0.9688, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 558/730, Speed: 5.20 secs/batch, TimeRemain: 0:14:54 || Loss: 0.0400, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 559/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 672/730, Speed: 5.13 secs/batch, TimeRemain: 0:04:57 || Loss: 0.0007, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 673/730, Speed: 5.09 secs/batch, TimeRemain: 0:04:50 || Loss: 0.0060, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 674/730, Speed: 5.19 secs/batch, TimeRemain: 0:04:50 || Loss: 0.0007, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 675/730, Speed: 5.07 secs/batch, TimeRemain: 0:04:39 || Loss: 0.0463, Acc: 0.9844, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 676/730, Speed: 5.19 secs/batch, TimeRemain: 0:04:40 || Loss: 0.0012, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 677/730, Speed: 5.13 secs/batch, TimeRemain: 0:04:31 || Loss: 0.0709, Acc: 0.9844, Rec: 0.6667730
, Epoc 5/5, BatchSz: 64 sents, Batch: 678/730, Speed: 5.14 secs/batch, TimeRemain: 0:04:27 || Loss: 0.0072, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 679/730, S

accuracy testing = 0.9473056197166443
recall testing = 0.09889749437570572
loss testing = 0.39300236105918884
>>>>>> Final Testing <<<<<<
>> ATTENTION weights:
################# Left - attention - weights 
[array([[-0.04486643,  0.0057978 , -0.00489222, ...,  0.00691065,
         0.03598357,  0.00890646],
       [ 0.03541507, -0.04945061, -0.01004615, ...,  0.02679747,
         0.03299609, -0.01381093],
       [-0.04160572,  0.02229307,  0.0402369 , ...,  0.02248259,
         0.01489022, -0.03911433],
       ...,
       [ 0.03403645, -0.02746133,  0.00132885, ...,  0.02098391,
        -0.04210263, -0.02199465],
       [ 0.01866749,  0.02331189,  0.00175661, ..., -0.02964565,
        -0.04474405, -0.02984447],
       [ 0.04631735, -0.01498282, -0.0359273 , ...,  0.01221329,
         0.02282301,  0.01391409]], dtype=float32), array([ 2.89987438e-05,  1.18494274e-04, -2.08908081e-04, ...,
       -1.56405687e-04, -2.92452278e-05,  6.17610785e-05], dtype=float32), array([ 0.17312463,  0.139

## Old one

In [33]:
from keras.layers import Concatenate

from keras.layers import Lambda
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, RepeatVector, Input, add, merge, Convolution1D, Convolution2D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.layers import concatenate
import numpy as np
import pdb
from nltk import tokenize
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold
import codecs
import gensim, sklearn
from string import punctuation
from collections import defaultdict
import sys
import math
import tensorflow as tf

from tabulate import tabulate

#from attention_lstm import AttentionLSTM
#from attention_lstm_without_weights import Attention
import time

trained_sample_handler = None
SAMPLE_TYPE_wiki, X_wiki, Y_wiki = None, None, None                 # Training + Development (need to split)
#SAMPLE_TYPE_wikitest, X_wikitest, Y_wikitest = None, None, None     # Testing
SAMPLE_TYPE_cli, X_cli, Y_cli = None, None, None              # Testing
SAMPLE_TYPE_bio, X_bio, Y_bio = None, None, None              # Testing
SAMPLE_TYPE_fic, X_fic, Y_fic = None, None, None              # Testing

GLOVE_EMBEDDING_DIM = 300
SCALE_LOSS_FUN = True

# SEQUENCES_LENGTH_FOR_TRAINING = 40

ONE_SIDE_CONTEXT_SIZE = 10

LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN = False
SAVE_MODEL_AFTER_EACH_EPOCH = True
TRAINABLE_EMBEDDINGS = False


np.random.seed(2345)   # IMP seed
saved_model_epoch_done = None

def load_saved_model():
    global saved_model_epoch_done
    print ("====== Loading the saved model =======")
    filename = sys.argv[1]
    epoch_done = filename.split(".")[0].split("_")[-1]
    print (">>> Continuing from epoch %d <<<<" %(int(epoch_done)+1))
    saved_model_epoch_done = int(epoch_done)

    model = load_model(filename)
    return model
        

def reshape(x):
      return tf.reshape(x, shape=[-1,2400])
        
        
def baseline_perceptron(sequence_len):

    print ('Build MAIN model...')
    #pdb.set_trace()
    main_input = Input(shape=(sequence_len,), dtype='float32', name='main-input')

    encode_1 = Dense(300, name='dense-1')(main_input)
    drop_out_1 = Dropout(0.3)(encode_1)

    encode_2 = Dense(300, name='dense-2')(drop_out_1)
    drop_out_2 = Dropout(0.3)(encode_2)

    encode_3 = Dense(300, name='dense-3')(drop_out_2)
    drop_out_3 = Dropout(0.3)(encode_3)

    output = Dense(2, activation='sigmoid')(drop_out_3)
    

    model = Model(input=[main_input], output=output)
    #model.compile(loss=w_binary_crossentropy, optimizer='rmsprop', metrics=['accuracy', 'recall'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall])

    print (model.summary(line_length=150, positions=[.46, .65, .77, 1.]))
    return model



def lstm_model(sequences_length_for_training, embedding_dim, embedding_matrix, vocab_size):

    which_model = 2
    # Look back is equal to -INF
    # This model creates a Stateful LSTM with lookback of the whole document
    # Input should be of the format (TOTAL_DOCUMENTS, TOTAL_SEQUENCES, SEQUENCE_DIM)
    # Also train using the custom trainer

    print ('Build MAIN model...')
    #pdb.set_trace()
    ngram_filters = [2, 3, 4, 5]
    conv_hidden_units = [200, 200, 200, 200]
    
    left_context= Input(shape=(ONE_SIDE_CONTEXT_SIZE+1, embedding_dim), dtype='float32', name='left-context')
    main_input = Input(shape=(1, embedding_dim), dtype='float32', name='main-input')
    right_context = Input(shape=(ONE_SIDE_CONTEXT_SIZE+1, embedding_dim), dtype='float32', name='right-context')

    context_embedder = TimeDistributed(Embedding(vocab_size + 1, GLOVE_EMBEDDING_DIM, weights=[embedding_matrix], input_length=embedding_dim, init='uniform'))
    main_input_embedder = TimeDistributed(Embedding(vocab_size + 1, GLOVE_EMBEDDING_DIM, weights=[embedding_matrix], input_length=embedding_dim, init='uniform'))
    embedded_input_left, embedded_input_main, embedded_input_right = context_embedder(left_context), main_input_embedder(main_input), context_embedder(right_context)

    convsL, convsM, convsR = [], [], []
    for n_gram, hidden_units in zip(ngram_filters, conv_hidden_units):
        conv_layer = Convolution1D(nb_filter=hidden_units,
                             filter_length=n_gram,
                             border_mode='same',
                             #border_mode='valid',
                             activation='tanh', name='Convolution-'+str(n_gram)+"gram")
        lef = TimeDistributed(conv_layer, name='TD-convolution-left-'+str(n_gram)+"gram")(embedded_input_left)
        mid = TimeDistributed(conv_layer, name='TD-convolution-mid-'+str(n_gram)+"gram")(embedded_input_main)
        rig = TimeDistributed(conv_layer, name='TD-convolution-right-'+str(n_gram)+"gram")(embedded_input_right)

        # Use Flatten() instead of MaxPooling()
        #flat_L = TimeDistributed(Flatten(), name='TD-flatten-left-'+str(n_gram)+"gram")(lef)
        #flat_M = TimeDistributed(Flatten(), name='TD-flatten-mid-'+str(n_gram)+"gram")(mid)
        #flat_R = TimeDistributed(Flatten(), name='TD-flatten-right-'+str(n_gram)+"gram")(rig)
        #convsL.append(flat_L), convsM.append(flat_M), convsR.append(flat_R)

        # Use GlobalMaxPooling1D() instead of Flatten()
        pool_L = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-left-'+str(n_gram)+"gram")(lef)
        pool_M = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-mid-'+str(n_gram)+"gram")(mid)
        pool_R = TimeDistributed(GlobalMaxPooling1D(), name='TD-GlobalMaxPooling-right-'+str(n_gram)+"gram")(rig)
        convsL.append(pool_L), convsM.append(pool_M), convsR.append(pool_R)
    
    convoluted_left=concatenate(convsL, axis=-1)  
    convoluted_mid=concatenate(convsM, axis=-1) 
    convoluted_right=concatenate(convsR, axis=-1) 
    CONV_DIM = sum(conv_hidden_units)
    #print(convoluted_left.shape)
    #print(convoluted_mid.shape)

    ####convoluted_mid, convoluted_left, convoluted_right, CONV_DIM = main_input, left_context, right_context, 300
    flat_mid = Flatten()(convoluted_mid)
    encode_mid = Dense(300, name='dense-intermediate-mid-encoder')(flat_mid)

    context_encoder_intermediate1 = Bidirectional(LSTM(600, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=True, stateful=False), name='BiLSTM-context-encoder-intermediate1', merge_mode='concat')
    #context_encoder_intermediate2 = Bidirectional(LSTM(500, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.1, dropout_U=0.1, return_sequences=True, stateful=False), name='BiLSTM-context-encoder-intermediate2', merge_mode='concat')
    context_encoder = Bidirectional(LSTM(600, input_shape=(ONE_SIDE_CONTEXT_SIZE, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=True, stateful=False), name='BiLSTM-context-encoder', merge_mode='concat')
    
    #context_encoder1_intermediate = Bidirectional(LSTM(600, input_shape=(1, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=False, stateful=False), name='BiLSTM-context-encoder1-intermediate', merge_mode='concat')
    #context_encoder1 = Bidirectional(LSTM(600, input_shape=(1, CONV_DIM), consume_less='gpu', dropout_W=0.3, dropout_U=0.3, return_sequences=False, stateful=False), name='BiLSTM-context-encoder1', merge_mode='concat')
    
    #encode_left = Attention(name='encode-left-attention')(context_encoder(context_encoder_intermediate1(convoluted_left)))
    #encode_right = Attention(name='encode-right-attention')(context_encoder(context_encoder_intermediate1(convoluted_right)))
    encode_left = AttentionWithContext(name='encode-left-attention')(context_encoder(context_encoder_intermediate1(convoluted_left)))
    encode_right = AttentionWithContext(name='encode-right-attention')(context_encoder(context_encoder_intermediate1(convoluted_right)))
    #encode_mid = AttentionWithContext(name='encode-mid-attention')(context_encoder1(convoluted_mid))
    ####encode_left = context_encoder(context_encoder_intermediate1(convoluted_left))
    ####encode_right = context_encoder(context_encoder_intermediate1(convoluted_right))
    print("encode-mid.shape",encode_mid.shape)
    encode_left_drop, encode_mid_drop, encode_right_drop = Dropout(0.3)(encode_left), Dropout(0.2)(encode_mid), Dropout(0.3)(encode_right)
    
    #encoded_info=concatenate([tf.reshape(encode_left_drop,[1,300]), encode_mid_drop, tf.reshape(encode_right_drop,[1,300])], axis=-1)
    #encode_left_drop=tf.reshape(encode_left_drop, shape=[-1, 300], name='encode_left_drop')
    #encode_right_drop=tf.reshape(encode_right_drop, shape=[-1, 300], name='encode_right_drop')
    #encode_left_drop = Lambda(reshape)(encode_left_drop)
    #encode_right_drop = Lambda(reshape)(encode_right_drop)
    print("sucessful")
    
    #XD = Lambda(concatenation,output_shape=(input_size,n+1))(encode_left_drop)
    encoded_info=Concatenate(axis=-1)([encode_left_drop,encode_right_drop])
    encoded_info = Lambda(reshape)(encoded_info)
    encoded_info=Concatenate(axis=-1)([encoded_info,encode_mid])
    #encoded_info = Merge(mode='concat', name='encode_info')([encode_left_drop, encode_mid_drop, encode_right_drop])

    decoded = Dense(500, name='decoded')(encoded_info)
    decoded_drop = Dropout(0.3, name='decoded_drop')(decoded)
    decoded1 = Dense(500, name='decoded1')(decoded_drop)
    decoded_drop1 = Dropout(0.3, name='decoded_drop1')(decoded1)
    output = Dense(1, activation='sigmoid')(decoded_drop1)
    model = Model(input=[left_context, main_input, right_context], output=output)
    #model = Model(input=[main_input], output=output)
    model.layers[1].trainable = TRAINABLE_EMBEDDINGS
    #model.compile(loss=w_binary_crossentropy, optimizer='rmsprop', metrics=['accuracy', 'recall'])
    model.compile(loss="binary_crossentropy", optimizer='adadelta', metrics=['accuracy', recall])


    print (model.summary(line_length=150, positions=[.46, .65, .77, 1.]))
    return model


def batch_gen_consecutive_context_segments_from_big_seq(typeOfBatch,X_with_doc, Y_with_doc, batch_size, one_side_context_size):
    # X should be (no_of_documents, total_sequences, bla, bla, bla ...)
    # Same should be Y
    X_Left_batch, Y_Left_batch, X_Right_batch, Y_Right_batch, X_Mid_batch, Y_Mid_batch = [], [], [], [], [], []
    for X, Y in zip(X_with_doc, Y_with_doc):   # iterate over document
        total_seq = X.shape[0]
        for i in range(total_seq):   # sample count
            # The neighbouring context do not contain the main sentence in their window
            #xL_i, xL_j, sent_i, xR_i, xR_j = i-one_side_context_size, i, i, i+1, i+one_side_context_size+1  # Example:: left: [0: 5] | Mid: [5: 6] | Right: [6: 11]
            #context_mat_size = one_side_context_size

            # The neighbouring context also contains the main sentence in the temporal data to derive some inference uing BLSTM
            xL_i, xL_j, sent_i, xR_i, xR_j = i-one_side_context_size, i+1, i, i, i+one_side_context_size+1  # Example:: left: [0: 6] | Mid: [5: 6] | Right: [5: 11]
            context_mat_size = one_side_context_size + 1

            if typeOfBatch == "train":    # Skip documents only when they are for training, for testing test on all the documents
                if total_seq < 2*one_side_context_size + 1:
                    print ("Too Small sequence: Found %d, required %d. DROPPING DOCUMENT" %(total_seq, 2*one_side_context_size+1))
                    break

            # Check if padding for the one_side_context_size required for both LEFT & RIGHT
            X_temp_Left, X_temp_Right, Y_temp_Left, Y_temp_Right = None, None, None, None
            if xL_i < 0:
                X_temp_Left, Y_temp_Left = np.array(X[0: xL_j]), np.array(Y[0: xL_j]) # Create resizable array not view
                #print "_________________"
                #print X_temp_Left.shape
                #print X_temp_Left
                shpX, shpY = list(X_temp_Left.shape), list(Y_temp_Left.shape)
                shpX[0] = shpY[0] = context_mat_size - xL_j
                X_temp_Left, Y_temp_Left = np.concatenate((np.zeros(shpX), X_temp_Left), axis=0), np.concatenate((np.zeros(shpY), Y_temp_Left), axis=0)
                #print X_temp_Left.shape
                #print X_temp_Left
                #print "-----------------"
            else:
                X_temp_Left, Y_temp_Left = X[xL_i: xL_j], Y[xL_i: xL_j]

            if xR_j > total_seq:
                X_temp_Right, Y_temp_Right = np.array(X[xR_i: total_seq]), np.array(Y[xR_i: total_seq]) # Create resizable array not view
                #print "+++++++++++++++++++++++++++"
                #print X_temp_Right.shape
                #print X_temp_Right
                shpX, shpY = list(X_temp_Right.shape), list(Y_temp_Right.shape)
                shpX[0] = shpY[0] = context_mat_size - (total_seq - xR_i)
                X_temp_Right, Y_temp_Right = np.concatenate((X_temp_Right, np.zeros(shpX)), axis=0), np.concatenate((Y_temp_Right, np.zeros(shpY)), axis=0)
                #print X_temp_Right
                #print X_temp_Right.shape
                #print "==========================="
            else:
                X_temp_Right, Y_temp_Right = X[xR_i: xR_j], Y[xR_i: xR_j]

            X_Left_batch.append(X_temp_Left), X_Right_batch.append(X_temp_Right)
            #Y_Left_batch.append(Y_Left_temp), Y_Right_batch.append(Y_Right_temp)       # No need for context groundtruths. May be we can test with that also later?
            X_Mid_batch.append(X[sent_i].reshape((1, -1))), Y_Mid_batch.append(Y[sent_i])  # X's should be TimeDistributed but not Y's
            if len(X_Left_batch) == batch_size:
                yield np.array(X_Left_batch), np.array(X_Mid_batch), np.asarray(X_Right_batch), np.array(Y_Mid_batch)
                X_Left_batch, Y_Left_batch, X_Right_batch, Y_Right_batch, X_Mid_batch, Y_Mid_batch = [], [], [], [], [], []
    if len(X_Left_batch):
        yield np.array(X_Left_batch), np.array(X_Mid_batch), np.asarray(X_Right_batch), np.array(Y_Mid_batch)


def batch_gen_sentences_without_context(X, Y, batch_size, fixed_size=False):
    # X should be (no_of_documents, total_sequences, bla, bla, bla ...)
    # Same should be Y
    # Specially for baselines_models
    total_seq = X.shape[0]
    for i in range(int(math.ceil(total_seq*1.0/batch_size))):   # sample count
        x_i, x_j = i*batch_size, (i+1)*batch_size
        X_temp, Y_temp = X[x_i: x_j], Y[x_i: x_j]
        count_sent = X_temp.shape[0]

        # Check if padding for the no of sentences required
        if fixed_size:
            if X_temp.shape[0] != batch_size:
                X_temp, Y_temp = np.array(X_temp), np.array(Y_temp) # Create resizable array not view
            shpX, shpY = list(X_temp.shape), list(Y_temp.shape)
            shpX[0], shpY[0] = batch_size, batch_size
            X_temp.resize(tuple(shpX)), Y_temp.resize(tuple(shpY))    # Resize to fit the batch size (ZEROES)

        yield np.array(X_temp), np.asarray(Y_temp)



def custom_fit(X_train, Y_train, X_test, Y_test, model, batch_size, epochs):

    # Print Train stats
    total_sentences, total_documents = 0, 0
    total_documents = X_train.shape[0]
    total_sentences = sum([doc.shape[0] for doc in X_train])
    print ("X-wiki TRAIN stats: Total %d sentences in %d documents" %(total_sentences, total_documents))

    class_weight = None
    if SCALE_LOSS_FUN:
        # Iterate as the no of sentences in each document is different
        # so np.unique() messes up.
        classes, counts = None, []
        for _temp_Yi in Y_train:
            classes, _temp_counts = np.unique(_temp_Yi, return_counts=True)
            counts.append(_temp_counts)
        counts = np.sum(counts, axis=0)
        class_weight = dict(zip(classes.tolist(), counts/float(sum(counts))))
        print (class_weight)

    train_avg_seg_len = np.mean([compute_avg_seg_len(Yi) for Yi in Y_train], axis=0)
    print (">> Train AVG_SEGMENT_LENGTH:", train_avg_seg_len)

    print ('Train...')
    start_epoch = 0
    if LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN:   # If we have saved model, then continue from the last epoch where we stopped
        start_epoch = saved_model_epoch_done  # The epoch count is zero indexed in TRAIN, while the count in saved file is 1 indexed

    for epoch in range(start_epoch, epochs):
        mean_tr_acc, mean_tr_loss, mean_tr_rec = [], [], []
        rLoss, rRecall, rAcc = 0,0,0 # Running parameters for printing while training
        for batch_count, (batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid) in enumerate(batch_gen_consecutive_context_segments_from_big_seq("train",X_train, Y_train, batch_size, ONE_SIDE_CONTEXT_SIZE)):
            #batch_Y_vec = to_categorical_MULTI_DIM(batch_Y, nb_classes=2)
                #batch_Y_mid = to_categorical(batch_Y_mid, num_classes=2) # Convert to output as 2 classes
                start = time.time()
                #print("batch_X_left",batch_X_left.shape)
                #print("batch_X_mid",batch_X_mid.shape)
                #print("batch_X_right", batch_X_right.shape)
                #print("batch_Y_mid", batch_Y_mid.shape)
                tr_loss, tr_acc, tr_rec = model.train_on_batch([batch_X_left, batch_X_mid, batch_X_right], batch_Y_mid)
                speed = time.time() - start
                mean_tr_acc.append(tr_acc)
                mean_tr_loss.append(tr_loss)
                mean_tr_rec.append(tr_rec)
                #rLoss, rRecall, rAcc = (rLoss*batch_count + tr_loss)/(batch_count + 1), (rRecall*batch_count + tr_rec)/(batch_count + 1), (rAcc*batch_count + tr_acc)/(batch_count + 1)
                #progbar.prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'rLoss': rLoss, 'rAcc': rAcc, 'rRec': rRecall })
                prog_bar(True, total_sentences, epochs, batch_size, epoch, batch_count, speed=speed, data={ 'Loss': tr_loss, 'Acc': tr_acc, 'Rec': tr_rec })
                # Print test results after every 100 batch trains
                
        if SAVE_MODEL_AFTER_EACH_EPOCH:
                model.save("model_trainable_%s_epoc_%d.h5" %(str(TRAINABLE_EMBEDDINGS), epoch+1))

        print (">> Epoch: %d/%d" %(epoch+1, epochs))
        print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
        print('recall training = {}'.format(np.mean(mean_tr_rec)))
        print('loss training = {}'.format(np.mean(mean_tr_loss)))

        #testing_on_data("Wikipedia(DEVELOPMENT)", X_test, Y_test, model, batch_size, summary_only=True)
        #save_predictions("Clinical", X_cli, Y_cli, model, batch_size, summary_only=True)
        #save_predictions("Biography", X_bio, Y_bio, model, batch_size, summary_only=True)
        #save_predictions("Fiction", X_fic, Y_fic, model, batch_size, summary_only=True)
        

        print('___________________________________')

   # Testing
    print ("####################################################################")
    print (">> (TEST) >> Testing, X:", X_test.shape, "Y:", Y_test.shape)
    mean_te_acc, mean_te_loss, mean_te_rec = [], [], []
    for i in range(X_test.shape[0]):
        X, Y = X_test[i], Y_test[i]
        X, Y = X.reshape((1,) + X.shape), Y.reshape((1,) + Y.shape)   # Convert to format of 1 document
        for batch_count, (batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid) in enumerate(batch_gen_consecutive_context_segments_from_big_seq("test",X, Y, batch_size, ONE_SIDE_CONTEXT_SIZE)):
            
            te_loss, te_acc, te_rec = model.test_on_batch([batch_X_left, batch_X_mid, batch_X_right], batch_Y_mid)
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
            mean_te_rec.append(te_rec)

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('recall testing = {}'.format(np.mean(mean_te_rec)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
        

def save_predictions(type_of_data, X_test, Y_test, model, batch_size, summary_only=False):
    # Predicting
    print ("====================== %s ======================" %(type_of_data))
    print ("GET PREDICTIONS... (SEPARATELY FOR EACH DOCUMENT)")
    data = {
        'wd_r': [],
        'wd_e': [],
        'pk': []
    }
    doc_idx = 18
    avg_segment_lengths_across_test_data = [] # Average segment length across the documents
    predictions_return = []
    
    for Xi_test, Yi_test in zip(X_test, Y_test):
        pred_per_doc = []
        Xi_test, Yi_test = Xi_test.reshape((1,) + Xi_test.shape), Yi_test.reshape((1,) + Yi_test.shape)   # Convert to format of 1 document
        for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq("test",Xi_test, Yi_test, batch_size, ONE_SIDE_CONTEXT_SIZE):
            batch_y_pred = model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right])
            pred_per_doc.append(batch_y_pred)


        #rounded = np.round(pred_per_doc)
    pred_per_doc = np.concatenate(pred_per_doc, axis=0)
        
    actual_avg_seg_length, result = windiff_and_pk_metric_ONE_SEQUENCE(Yi_test[0], pred_per_doc, window_size=-1, rounded=False, print_individual_stats=not summary_only)
    avg_segment_lengths_across_test_data.append(actual_avg_seg_length)
    data['pk'].append(result['pk'])
    data['wd_r'].append(result['wd_r'])
    data['wd_e'].append(result['wd_e'])

    print (">> Summary (%s):" %(type_of_data))
    print ("AVG segment length in test data: %f, std: %f" % (np.mean(avg_segment_lengths_across_test_data), np.std(avg_segment_lengths_across_test_data)))
    print ("WinDiff metric (Windiff_r):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_r']), np.std(data['wd_r']), np.min(data['wd_r']), np.max(data['wd_r'])))
    print ("WinDiff metric (Windiff_e):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_e']), np.std(data['wd_e']), np.min(data['wd_e']), np.max(data['wd_e'])))
    print ("Pk metric:: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['pk']), np.std(data['pk']), np.min(data['pk']), np.max(data['pk'])))
    print('___________________________________')



def testing_on_data(type_of_data, X_test, Y_test, model, batch_size, summary_only=False, visualize=False):
    # Predicting
    print ("====================== %s ======================" %(type_of_data))
    print ("Predicting... (SEPARATELY FOR EACH DOCUMENT)")
    data = {
        'wd_r': [],
        'wd_e': [],
        'pk': []
    }
    avg_segment_lengths_across_test_data = [] # Average segment length across the documents
    for Xi_test, Yi_test in zip(X_test, Y_test):
        pred_per_doc = []
        #Xi_test, Yi_test = Xi_test.reshape((1,) + Xi_test.shape), Yi_test.reshape((1,) + Yi_test.shape)   # Convert to format of 1 document
        for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq(Xi_test, Yi_test, batch_size, ONE_SIDE_CONTEXT_SIZE):
            batch_y_pred = model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right])
            pred_per_doc.append(batch_y_pred)

        if not len(pred_per_doc): # batch generator might drop a few documents
            continue

        #rounded = np.round(pred_per_doc)
        pred_per_doc = np.concatenate(pred_per_doc, axis=0)
        actual_avg_seg_length, result = windiff_and_pk_metric_ONE_SEQUENCE(Yi_test[0], pred_per_doc, window_size=-1, rounded=False, print_individual_stats=not summary_only)
        avg_segment_lengths_across_test_data.append(actual_avg_seg_length)
        data['pk'].append(result['pk'])
        data['wd_r'].append(result['wd_r'])
        data['wd_e'].append(result['wd_e'])

    print (">> Summary (%s):" %(type_of_data))
    print ("AVG segment length in test data: %f" % (np.mean(avg_segment_lengths_across_test_data)))
    print ("WinDiff metric (Windiff_r):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_r']), np.std(data['wd_r']), np.min(data['wd_r']), np.max(data['wd_r'])))
    print ("WinDiff metric (Windiff_e):: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['wd_e']), np.std(data['wd_e']), np.min(data['wd_e']), np.max(data['wd_e'])))
    print ("Pk metric:: avg: %f | std: %f | min: %f | max: %f" %(np.mean(data['pk']), np.std(data['pk']), np.min(data['pk']), np.max(data['pk'])))
    print('___________________________________')




def split_data(X, Y, train_split):
    samples = X.shape[0]
    train_samples = int(X.shape[0]*train_split)
    X_train, Y_train = X[:train_samples+1,], Y[:train_samples+1,]
    X_test, Y_test = X[train_samples+1:,], Y[train_samples+1:,]
    print ("X_train:", X_train.shape, " | Y_train:", Y_train.shape)
    print ("X_test:", X_test.shape, " | Y_test:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test


def train_model(X, Y, model, train_split=0.8, epochs=5, batch_size=32):
    global X_wiki, Y_wiki, X_cli, Y_cli, X_bio, Y_bio, X_fic, Y_fic

    which_model = 2

    # This is only for training! (If train_split =1 then only TEST)
    X_train, Y_train, X_test, Y_test = split_data(X, Y, train_split=train_split)

    if which_model == 2:
        #custom_fit(X_train, Y_train, X_test, Y_test, model=model, batch_size=batch_size, epochs=epochs)
        custom_fit(X_train, Y_train, X_test, Y_test, model=model, batch_size=batch_size, epochs=epochs)
        
        print (">>>>>> Final Testing <<<<<<")
        print (">> ATTENTION weights:")
        print ("################# Left - attention - weights ")
        attn_weights_left = [model.get_layer("encode-left-attention").get_weights(), model.get_layer("encode-left-attention").get_weights()]
        print (attn_weights_left[0])
        print (attn_weights_left[1])
          
        attn_weights_right = [model.get_layer("encode-right-attention").get_weights(), model.get_layer("encode-right-attention").get_weights()]
        print ("################# RIght - attention - weights ")
        print (attn_weights_right[0])
        print (attn_weights_right[1])

        #testing_on_data("Wikipedia", X_test, Y_test, model, batch_size, summary_only=True)
        #testing_on_data("Clinical", X_cli, Y_cli, model, batch_size)
        #testing_on_data("Biography", X_bio, Y_bio, model, batch_size)
        #testing_on_data("Fiction", X_fic, Y_fic, model, batch_size, summary_only=True)

    
    
if __name__ == "__main__":

    # Print parameters
    print ("=== SCALE_LOSS_FUN: %d, ONE_SIDE_CONTEXT_SIZE: %d ===" % (int(SCALE_LOSS_FUN), ONE_SIDE_CONTEXT_SIZE))
    print ("NOTE: Make sure you have MIN_SENTENCES_IN_DOCUMENT >= 2*context_size + 1")

    # For which_model == 2
    SAMPLE_TYPE_wiki, X_wiki, Y_wiki, trained_sample_handler = get_input(sample_type=7, shuffle_documents=True, pad=False)
    NO_OF_SAMPLES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = X_wiki.shape[0], -1, X_wiki[0].shape[1]          #MAX_SEQUENCE_LENGTH is is already padded
    
    # For which_model == 2
    # Biography data for training
    SAMPLE_TYPE_bio, X_bio, Y_bio, trained_sample_handler = get_input(sample_type=5, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    NO_OF_SAMPLES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = X_bio.shape[0], -1, X_bio[0].shape[1]          #MAX_SEQUENCE_LENGTH is is already padded
    
    # Only for testing
    SAMPLE_TYPE_cli, X_cli, Y_cli, trained_sample_handler = get_input(sample_type=4, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    SAMPLE_TYPE_fic, X_fic, Y_fic, trained_sample_handler = get_input(sample_type=6, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    #SAMPLE_TYPE_wikitest, X_wikitest, Y_wikitest, trained_sample_handler = get_input(sample_type=7, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
    
    #print "Check data type"

    sequence_length = X_wiki[0].shape[-1]
    dictionary_object = trained_sample_handler.dictionary
    embedding_W = dictionary_object.get_embedding_weights()

    # COmment temporarily for context variation experiemnt
    if LOAD_SAVED_MODEL_AND_CONTINUE_TRAIN:
        model = load_saved_model()
    else:
        #model = baseline_perceptron(sequence_length)
        model = lstm_model(-1, EMBEDDING_DIM, embedding_W, len(dictionary_object.word2id_dic))

    train_model(X_wiki, Y_wiki, model, train_split=0.8, epochs=5, batch_size=64)
    ##train_LSTM(X_bio, Y_bio, model, embedding_W, train_split=0.7, epochs=1, batch_size=32)




=== SCALE_LOSS_FUN: 1, ONE_SIDE_CONTEXT_SIZE: 10 ===
NOTE: Make sure you have MIN_SENTENCES_IN_DOCUMENT >= 2*context_size + 1
Total WIKIPEDIA(test) data: 300 chapters, 2265 paragraphs and 58578 sentences
Samples Loading took 1.2916810512542725 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 81.838045835495
Converting doc to martices 51 and the time passed is: 109.65490198135376
Converting doc to martices 101 and the time passed is: 139.9914710521698
Converting doc to martices 151 and the time passed is: 172.7554919719696
Converting doc to martices 201 and the time passed is: 202.00729703903198
Converting doc to martices 251 and the time passed is: 231.05257320404053
Total samples: 300
SHUFFLE: Shuffled input document order! (X: (300,) , Y: (300,) )
Total BIOGRAPHY data: 11 chapters, 298 paragraphs and 2285 sentences
Samples Loading took 0.08951210975646973 seconds
Converting doc to martices 1 and the time passed is: 0.09016227722167969


/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(80472, 300, weights=[array([[ ..., input_length=20, embeddings_initializer="uniform")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(80472, 300, weights=[array([[ ..., input_length=20, embeddings_initializer="uniform")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-2gram", filters=200, kernel_size=2, padding="same")`
/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:122: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-3gram", filters=200, kernel_size=3, padding="same")`
/Users/monaafadavi/anaconda3/l

encode-mid.shape (?, 300)
sucessful


/Users/monaafadavi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


______________________________________________________________________________________________________________________________________________________
Layer (type)                                                         Output Shape                Param #           Connected to                       
left-context (InputLayer)                                            (None, 11, 20)              0                                                    
______________________________________________________________________________________________________________________________________________________
right-context (InputLayer)                                           (None, 11, 20)              0                                                    
______________________________________________________________________________________________________________________________________________________
time_distributed_15 (TimeDistributed)                                (None, 11, 20, 300)      

...

, Epoc 5/5, BatchSz: 64 sents, Batch: 332/730, Speed: 4.84 secs/batch, TimeRemain: 0:32:04 || Loss: 0.0001, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 333/730, Speed: 4.84 secs/batch, TimeRemain: 0:32:01 || Loss: 0.0000, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 334/730, Speed: 4.87 secs/batch, TimeRemain: 0:32:07 || Loss: 0.0000, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 335/730, Speed: 4.84 secs/batch, TimeRemain: 0:31:50 || Loss: 0.0000, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 336/730, Speed: 4.85 secs/batch, TimeRemain: 0:31:51 || Loss: 0.0000, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 337/730, Speed: 4.91 secs/batch, TimeRemain: 0:32:09 || Loss: 0.0000, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 338/730, Speed: 4.87 secs/batch, TimeRemain: 0:31:47 || Loss: 0.0007, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 339/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 452/730, Speed: 4.87 secs/batch, TimeRemain: 0:22:34 || Loss: 0.0042, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 453/730, Speed: 4.99 secs/batch, TimeRemain: 0:23:02 || Loss: 0.2508, Acc: 0.9688, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 454/730, Speed: 4.85 secs/batch, TimeRemain: 0:22:18 || Loss: 0.0002, Acc: 1.0000, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 455/730, Speed: 4.94 secs/batch, TimeRemain: 0:22:37 || Loss: 0.0296, Acc: 0.9844, Rec: 0.8000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 456/730, Speed: 4.96 secs/batch, TimeRemain: 0:22:40 || Loss: 0.0068, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 457/730, Speed: 4.84 secs/batch, TimeRemain: 0:22:00 || Loss: 0.0140, Acc: 0.9844, Rec: 0.5000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 458/730, Speed: 4.94 secs/batch, TimeRemain: 0:22:23 || Loss: 0.0131, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 459/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 572/730, Speed: 4.97 secs/batch, TimeRemain: 0:13:05 || Loss: 0.3591, Acc: 0.9531, Rec: 0.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 573/730, Speed: 5.11 secs/batch, TimeRemain: 0:13:22 || Loss: 0.0842, Acc: 0.9688, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 574/730, Speed: 4.86 secs/batch, TimeRemain: 0:12:38 || Loss: 0.0080, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 575/730, Speed: 4.96 secs/batch, TimeRemain: 0:12:48 || Loss: 0.0046, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 576/730, Speed: 4.89 secs/batch, TimeRemain: 0:12:32 || Loss: 0.0075, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 577/730, Speed: 4.85 secs/batch, TimeRemain: 0:12:21 || Loss: 0.0473, Acc: 0.9844, Rec: 0.6667730
, Epoc 5/5, BatchSz: 64 sents, Batch: 578/730, Speed: 4.85 secs/batch, TimeRemain: 0:12:16 || Loss: 0.0042, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 579/730, S

, Epoc 5/5, BatchSz: 64 sents, Batch: 692/730, Speed: 4.87 secs/batch, TimeRemain: 0:03:05 || Loss: 0.0007, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 693/730, Speed: 4.93 secs/batch, TimeRemain: 0:03:02 || Loss: 0.0009, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 694/730, Speed: 4.90 secs/batch, TimeRemain: 0:02:56 || Loss: 0.0052, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 695/730, Speed: 4.82 secs/batch, TimeRemain: 0:02:48 || Loss: 0.0004, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 696/730, Speed: 4.88 secs/batch, TimeRemain: 0:02:45 || Loss: 0.0071, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 697/730, Speed: 4.91 secs/batch, TimeRemain: 0:02:42 || Loss: 0.0081, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 698/730, Speed: 4.93 secs/batch, TimeRemain: 0:02:37 || Loss: 0.0004, Acc: 1.0000, Rec: 1.0000730
, Epoc 5/5, BatchSz: 64 sents, Batch: 699/730, S

################# RIght - attention - weights 
[array([[-0.00785891,  0.04900171,  0.0246044 , ..., -0.02996135,
        -0.01027044, -0.03676203],
       [-0.02339327, -0.02215697, -0.04505354, ...,  0.05002444,
         0.04687396,  0.02657745],
       [ 0.004025  ,  0.00590559,  0.01366511, ..., -0.03088457,
        -0.02328431,  0.03910632],
       ...,
       [-0.00086838, -0.01726245,  0.01099063, ...,  0.01767307,
        -0.01079839,  0.04606966],
       [ 0.00155261,  0.02529302, -0.01599226, ..., -0.00987943,
         0.02441586,  0.04991155],
       [-0.01333287, -0.02177794,  0.04266092, ..., -0.0386062 ,
        -0.02016717,  0.03744433]], dtype=float32), array([ 5.1815965e-05, -8.6398468e-05,  1.5171352e-05, ...,
       -5.4209994e-04, -1.8024142e-04, -6.9416075e-07], dtype=float32), array([ 0.27553222, -0.23092653, -0.16519244, ..., -0.2817269 ,
       -0.256694  , -0.03756207], dtype=float32)]
[array([[-0.00785891,  0.04900171,  0.0246044 , ..., -0.02996135,
        -0.

## Building New_model by getting rid of last layers of pretraind model
I aim to get rid of the two last dense and softmax layers and have 2700 dimension representation of each sentence which is the concatenation of left-attention, right_attention, and mid_sentence (concatenate_10 (Concatenate))

In [49]:
new_model = Model(model.inputs, model.layers[-6].output)
new_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left-context (InputLayer)       (None, 11, 20)       0                                            
__________________________________________________________________________________________________
right-context (InputLayer)      (None, 11, 20)       0                                            
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 11, 20, 300)  24141600    left-context[0][0]               
                                                                 right-context[0][0]              
__________________________________________________________________________________________________
TD-convolution-left-2gram (Time (None, 11, 20, 200)  120200      time_distributed_3[0][0]         
__________

## Getting 2700 fatures for every sequence of sentences,


In [102]:
SAMPLE_TYPE_wiki, X_wiki, Y_wiki, trained_sample_handler = get_input(sample_type=7, shuffle_documents=True, pad=False)
NO_OF_SAMPLES, MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = X_wiki.shape[0], -1, X_wiki[0].shape[1]
X_train, Y_train, X_test, Y_test = split_data(X_wiki, Y_wiki, train_split=0.8)

for Xi_test, Yi_test in zip(X_test, Y_test):
        pred_per_doc = []
        Xi_test, Yi_test = Xi_test.reshape((1,) + Xi_test.shape), Yi_test.reshape((1,) + Yi_test.shape)   # Convert to format of 1 document
        for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq("test",Xi_test, Yi_test, 10, 10):
            output = new_model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right])
            print(output.shape)
            print(output)

Total WIKIPEDIA(test) data: 300 chapters, 2265 paragraphs and 58578 sentences
Samples Loading took 0.3029658794403076 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 60.785741090774536
Converting doc to martices 51 and the time passed is: 89.83574891090393
Converting doc to martices 101 and the time passed is: 122.15359091758728
Converting doc to martices 151 and the time passed is: 156.35360097885132
Converting doc to martices 201 and the time passed is: 186.5855040550232
Converting doc to martices 251 and the time passed is: 216.694669008255
Total samples: 300
SHUFFLE: Shuffled input document order! (X: (300,) , Y: (300,) )
X_train: (241,)  | Y_train: (241,)
X_test: (59,)  | Y_test: (59,)
(10, 2700)
[[ 0.02306508 -0.0916946  -0.03603848 ...  0.02700989  0.2565235
   0.04666484]
 [ 0.03461283 -0.07254488 -0.00552892 ... -0.07614648 -0.12593777
   0.10593075]
 [ 0.03785278 -0.06343061  0.01211508 ... -0.07150396 -0.2087029
   0.09417948

(10, 2700)
[[ 3.8304480e-03  6.1873421e-03  2.6806450e-02 ...  5.6799714e-02
  -2.6146388e-01  8.3039224e-02]
 [ 7.4207899e-05  9.2722531e-03  2.8793201e-02 ... -2.9793825e-02
  -2.5749972e-01  1.3762662e-01]
 [-1.5597222e-03  1.0148926e-02  2.9066036e-02 ... -4.7589052e-02
  -9.2378482e-02  2.6726339e-02]
 ...
 [-3.3035572e-03  6.6914260e-03  2.0539314e-02 ... -2.6589807e-02
  -1.7932814e-01 -1.4740434e-03]
 [-7.8753261e-03  4.7708908e-03  2.4466183e-02 ... -1.1445135e-02
  -2.6854101e-01  1.2862474e-01]
 [-1.1660811e-02  3.9732810e-03  2.7866516e-02 ... -6.8066195e-02
  -1.8173449e-01  7.2616786e-02]]
(10, 2700)
[[-0.01410192  0.00558026  0.03124704 ...  0.01294887 -0.06563354
   0.04099652]
 [-0.01031609  0.00620337  0.03178063 ... -0.00425199 -0.1412775
   0.12501392]
 [-0.00775634  0.00873331  0.03664336 ... -0.05737625 -0.09471828
   0.1301905 ]
 ...
 [ 0.0009504   0.01226327  0.03169159 ... -0.05593464 -0.05372589
   0.04725764]
 [-0.00029983  0.01072663  0.02926229 ... -0.02453

(10, 2700)
[[ 5.00240270e-03  1.39426803e-02  4.93294746e-02 ... -7.56455660e-02
  -1.59936234e-01  1.52247116e-01]
 [ 5.90426754e-03  1.39534548e-02  4.92543019e-02 ... -7.89169222e-02
  -1.63507015e-02  3.98912951e-02]
 [ 4.42859996e-03  1.31495465e-02  4.53910418e-02 ... -5.49088418e-02
  -5.03774062e-02  7.69502819e-02]
 ...
 [ 7.55273504e-07  1.39406146e-02  2.60837544e-02 ... -8.62501860e-02
  -1.70140296e-01  8.45800787e-02]
 [-8.42330279e-04  8.44645314e-03  2.02663708e-02 ...  3.39261442e-02
  -1.08220756e-01  8.53982568e-02]
 [-5.56078879e-03  7.31142424e-03  1.85424481e-02 ... -1.01261660e-02
  -3.40135038e-01  1.39006928e-01]]
(10, 2700)
[[-0.00809338  0.00570545  0.02414941 ... -0.08260816 -0.38548744
   0.18026386]
 [-0.00441139  0.00563469  0.0226721  ... -0.05056062 -0.05801408
   0.05779979]
 [-0.00103267  0.00459362  0.02707764 ... -0.04959562 -0.09379865
   0.10690244]
 ...
 [ 0.00386636  0.00097553  0.02749751 ... -0.06654599 -0.10309549
   0.08828767]
 [ 0.00143191

(10, 2700)
[[-0.00752878  0.01375671  0.03487474 ...  0.03407101 -0.0141618
   0.09684171]
 [-0.00711163  0.01259236  0.0311542  ... -0.02262349 -0.3257689
   0.18989275]
 [-0.00376522  0.01588352  0.03829409 ... -0.03756429 -0.07302709
   0.07700303]
 ...
 [ 0.00546463  0.01738392  0.04485002 ... -0.07228646 -0.26485482
   0.08711064]
 [ 0.00409796  0.01780705  0.04207126 ... -0.01639082 -0.04768556
   0.06588945]
 [ 0.00219126  0.01498095  0.03691984 ... -0.05944801 -0.25978792
   0.10094546]]
(10, 2700)
[[ 8.1036938e-04  1.5142266e-02  3.5556577e-02 ... -1.1215167e-01
  -6.4899772e-02  3.1582229e-02]
 [ 8.8795823e-05  1.3107785e-02  3.0503448e-02 ... -6.1726958e-02
  -3.9681923e-03  3.5656247e-02]
 [-5.7821703e-04  9.5742038e-03  2.4035698e-02 ... -1.3274446e-02
  -1.2094453e-01  1.1334778e-01]
 ...
 [ 7.0325364e-03  9.4186924e-03  4.2307884e-02 ... -3.1801358e-02
  -1.4960824e-01  8.6160854e-02]
 [ 5.4529510e-03  7.9892166e-03  3.9281014e-02 ... -5.7083439e-02
  -1.3975489e-01  7.7

(10, 2700)
[[ 0.01105843  0.00944342  0.03150782 ...  0.03706709 -0.17260803
   0.07332562]
 [ 0.00859779  0.00987334  0.02874436 ... -0.01810313 -0.16306572
   0.11033036]
 [ 0.00645205  0.0125664   0.03073952 ... -0.0631799  -0.08563589
   0.0838459 ]
 ...
 [ 0.00828976  0.01212736  0.0355127  ... -0.02769946 -0.07209871
   0.10987869]
 [ 0.0040416   0.01451731  0.03879431 ... -0.02527691 -0.14318866
  -0.0123195 ]
 [ 0.00267944  0.01513842  0.03991847 ... -0.07253578 -0.13764334
   0.04739109]]
(10, 2700)
[[ 3.8404644e-03  1.7255001e-02  4.0777065e-02 ... -5.7438605e-02
  -1.5983385e-01  4.5101248e-02]
 [ 5.9992508e-03  1.7005045e-02  4.2426083e-02 ... -2.0444337e-02
  -1.5562496e-01  1.5079862e-01]
 [ 4.7847372e-03  1.6334392e-02  4.4242598e-02 ...  2.1138426e-02
  -4.2336427e-02  8.2419768e-02]
 ...
 [ 5.1124962e-03  1.7809255e-02  4.0451024e-02 ...  8.7254703e-02
  -1.8025126e-01  1.4707386e-01]
 [ 5.2679079e-03  1.7255608e-02  4.0116638e-02 ... -1.6220856e-02
  -8.7648004e-02  7

...

(10, 2700)
[[-0.0050994   0.00612621  0.02827889 ...  0.0041237   0.00535448
   0.09028411]
 [-0.00449771  0.0046406   0.02437334 ... -0.05860311 -0.22611962
   0.064614  ]
 [-0.00274558  0.00149551  0.02130419 ... -0.04972326 -0.12404309
   0.04230784]
 ...
 [ 0.00424546 -0.00331758  0.0176022  ...  0.01526723 -0.28038
   0.1376236 ]
 [ 0.00624331 -0.00471099  0.01880257 ... -0.03582923 -0.19450127
   0.13139118]
 [ 0.00725447 -0.00494987  0.01825195 ... -0.02355826 -0.12986259
   0.06996939]]
(10, 2700)
[[ 0.00935956 -0.00241367  0.02701567 ... -0.02059028 -0.12716135
   0.0793018 ]
 [ 0.01573047 -0.0029658   0.02948024 ... -0.02176797 -0.07306728
   0.07137437]
 [ 0.01148748 -0.00100687  0.0339896  ... -0.03183047 -0.03892583
   0.04555611]
 ...
 [ 0.00770034  0.00714577  0.04502909 ... -0.02387573 -0.04410876
   0.06550528]
 [ 0.0072193   0.00890842  0.0415741  ...  0.043605   -0.14693893
   0.04470259]
 [ 0.00643692  0.00883699  0.03998137 ... -0.04985115 -0.06552906
   0.11942725

(10, 2700)
[[ 0.00897582  0.0130258   0.04236995 ... -0.03382247 -0.09471181
   0.09976816]
 [ 0.00885639  0.0141324   0.04228413 ... -0.05561407 -0.07259577
   0.11677267]
 [ 0.00580479  0.01405868  0.0387453  ... -0.07018778 -0.0608964
   0.13238563]
 ...
 [ 0.00198148  0.01134932  0.03451977 ...  0.00336825 -0.11118636
   0.07295731]
 [ 0.00371777  0.00907631  0.03679518 ... -0.00140297 -0.03670075
   0.10740158]
 [ 0.00738765  0.00864969  0.04071828 ... -0.03722384 -0.06758554
   0.05935566]]
(10, 2700)
[[ 0.00888066  0.00878809  0.04075159 ... -0.03392208 -0.10927404
   0.09969382]
 [ 0.00667551  0.00715928  0.03859435 ... -0.04017797 -0.06701819
   0.11871374]
 [ 0.00775488  0.00588722  0.03906947 ... -0.0711527  -0.16839238
   0.06913872]
 ...
 [ 0.00877606  0.00842838  0.03401072 ... -0.00446229 -0.21281613
   0.1219307 ]
 [ 0.00768803  0.00849395  0.03519777 ...  0.01994094 -0.09372495
   0.07917272]
 [ 0.00709971  0.00863639  0.03616025 ... -0.0425943  -0.41387945
   0.141947

(10, 2700)
[[ 0.01613928  0.01552967  0.04247495 ... -0.02523242 -0.02486475
   0.04857771]
 [ 0.01536806  0.01325789  0.03915602 ... -0.0763936  -0.13235235
   0.02934773]
 [ 0.01250054  0.01414908  0.03682703 ... -0.00364235 -0.08143093
   0.07995468]
 ...
 [ 0.00427475  0.0170513   0.03761306 ...  0.02265087 -0.1606287
   0.10286039]
 [ 0.00332286  0.01846601  0.03858852 ... -0.01814558 -0.03938945
   0.09795323]
 [ 0.00286476  0.01764814  0.03925053 ... -0.06443261 -0.11923227
   0.04761537]]
(10, 2700)
[[ 0.00179112  0.01883565  0.04107258 ... -0.03880439 -0.11178312
   0.06710422]
 [ 0.00229097  0.01971894  0.04447251 ... -0.05679261 -0.16502625
   0.12458305]
 [ 0.00333336  0.0201451   0.04582408 ... -0.04111835 -0.10728016
   0.12120785]
 ...
 [ 0.01265797  0.01452867  0.04775513 ... -0.06180057 -0.08323202
   0.10826668]
 [ 0.01189854  0.01576015  0.04797784 ... -0.01483551 -0.18094128
   0.11745067]
 [ 0.01180786  0.01546547  0.04679566 ... -0.03085164 -0.13780905
   0.155207

(10, 2700)
[[ 0.0106545   0.01040627  0.02921811 ...  0.03610089 -0.10486044
   0.05913752]
 [ 0.00990469  0.00947994  0.02868878 ... -0.08288568 -0.0518048
   0.08419326]
 [ 0.01070944  0.00984349  0.02894304 ... -0.02180258 -0.03256905
   0.05079536]
 ...
 [ 0.01126263  0.01108279  0.03519612 ... -0.05813663 -0.03990215
   0.09781693]
 [ 0.01071651  0.01028317  0.03803255 ... -0.06227263 -0.07866901
   0.07412398]
 [ 0.01343501  0.01042165  0.04157778 ... -0.01549302 -0.13521849
   0.0981753 ]]
(10, 2700)
[[ 0.01670916  0.00877425  0.04178773 ...  0.00577257 -0.06813984
   0.07650691]
 [ 0.0200072   0.00605352  0.04216103 ... -0.00414458 -0.04379629
   0.0344885 ]
 [ 0.01908147  0.00615482  0.04159686 ... -0.05352385  0.01112117
   0.02130681]
 ...
 [ 0.01936875  0.00696441  0.03693007 ... -0.04648979  0.02841265
   0.0376624 ]
 [ 0.01788897  0.00648124  0.03405531 ... -0.03481212 -0.11437941
   0.05893555]
 [ 0.01683475  0.00587408  0.03193289 ... -0.04846942 -0.03757609
   0.053646

(10, 2700)
[[ 0.00242761  0.01639662  0.04450327 ... -0.05435721 -0.16136096
   0.07961996]
 [-0.00051091  0.01690952  0.04478601 ... -0.01656837 -0.3236137
   0.10061771]
 [ 0.00070748  0.01458367  0.03912408 ... -0.00408339 -0.08066252
   0.0668295 ]
 ...
 [ 0.00269326  0.00192077  0.03535974 ... -0.04221163 -0.06868285
   0.07598037]
 [ 0.00775123  0.00185346  0.03932345 ...  0.0018732  -0.00064894
   0.05889786]
 [ 0.00815762  0.00033037  0.03674773 ... -0.01022977 -0.20322937
   0.09711042]]
(10, 2700)
[[ 0.00567709  0.00266279  0.03428116 ... -0.05956614 -0.02454407
   0.09859596]
 [ 0.00622301  0.00469291  0.03457554 ... -0.05925902 -0.13258056
   0.08813372]
 [ 0.00433639  0.00331944  0.02958399 ...  0.0100614  -0.1021733
   0.17262201]
 ...
 [-0.00381862  0.00588424  0.04587343 ... -0.03743886 -0.06478463
   0.09067303]
 [-0.00242481  0.00565989  0.04749998 ... -0.06203505 -0.14831121
   0.13028063]
 [-0.0005535   0.00466051  0.04435795 ... -0.0600449  -0.14636849
   0.1026585

## Pointer Network : 
A multi-layer Pointer Network.  Pointer Networks is a new neural architecture to learn the conditional probability of an output sequence with elements that are discrete tokens corresponding to positions in an input sequence.

In [158]:
import tensorflow as tf

START_ID=0
PAD_ID=1
END_ID=2

class PointerWrapper(tf.contrib.seq2seq.AttentionWrapper):
  """Customized AttentionWrapper for PointerNet."""

  def __init__(self,cell,attention_size,memory,initial_cell_state=None,name=None):
    
    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(attention_size, memory, probability_fn=lambda x: x )
    
    cell_input_fn=lambda input, attention: input
    # Call super __init__
    super(PointerWrapper, self).__init__(cell,
                                         attention_mechanism=attention_mechanism,
                                         attention_layer_size=None,
                                         alignment_history=False,
                                         cell_input_fn=cell_input_fn,
                                         output_attention=True,
                                         initial_cell_state=initial_cell_state,
                                         name=name)
  @property
  def output_size(self):
    return self.state_size.alignments

  def call(self, inputs, state):
    _, next_state = super(PointerWrapper, self).call(inputs, state)
    return next_state.alignments, next_state
 

class PointerNet(object):



  def __init__(self, batch_size=128, max_input_sequence_len=10, max_output_sequence_len=5, 
              rnn_size=128, attention_size=128, num_layers=2, beam_width=2,
              learning_rate=0.001, max_gradient_norm=5, forward_only=False):
    """Create the model.
    Args:
      batch_size: the size of batch during training
      max_input_sequence_len: the maximum input length 
      max_output_sequence_len: the maximum output length
      rnn_size: the size of each RNN hidden units
      attention_size: the size of dimensions in attention mechanism
      num_layers: the number of stacked RNN layers
      beam_width: the width of beam search 
      learning_rate: the initial learning rate during training
      max_gradient_norm: gradients will be clipped to maximally this norm.
      forward_only: whether the model is forwarding only
    """
    #print("I am here")
    self.batch_size = batch_size
    self.max_input_sequence_len = max_input_sequence_len
    self.max_output_sequence_len = max_output_sequence_len
    self.forward_only = forward_only
    self.init_learning_rate = learning_rate
    # Note we have three special tokens namely 'START', 'PAD' and 'END'
    # Here the size of vocab need be added by 3
    self.vocab_size = max_input_sequence_len+3
    # Global step
    self.global_step = tf.Variable(0, trainable=False)
    # Choose LSTM Cell
    cell = tf.contrib.rnn.LSTMCell
    # Create placeholders
    self.inputs = tf.placeholder(tf.float32, shape=[self.batch_size,self.max_input_sequence_len,2700], name="inputs")
    self.outputs = tf.placeholder(tf.int32, shape=[self.batch_size,self.max_output_sequence_len+1], name="outputs")
    self.enc_input_weights = tf.placeholder(tf.int32,shape=[self.batch_size,self.max_input_sequence_len], name="enc_input_weights")
    self.dec_input_weights = tf.placeholder(tf.int32,shape=[self.batch_size,self.max_output_sequence_len], name="dec_input_weights")
    # Calculate the lengths
    enc_input_lens=tf.reduce_sum(self.enc_input_weights,axis=1)
    dec_input_lens=tf.reduce_sum(self.dec_input_weights,axis=1)
    # Special token embedding
    special_token_embedding = tf.get_variable("special_token_embedding", [3,2700], tf.float32, tf.contrib.layers.xavier_initializer())
    print( special_token_embedding)
    # Embedding_table
    # Shape: [batch_size,vocab_size,features_size]
    embedding_table = tf.concat([tf.tile(tf.expand_dims(special_token_embedding,0),[self.batch_size,1,1]), self.inputs],axis=1)   
    # Unstack embedding_table
    # Shape: batch_size*[vocab_size,features_size]
    embedding_table_list = tf.unstack(embedding_table, axis=0)
    # Unstack outputs
    # Shape: (max_output_sequence_len+1)*[batch_size]
    outputs_list = tf.unstack(self.outputs, axis=1)
    # targets
    # Shape: [batch_size,max_output_sequence_len]
    self.targets = tf.stack(outputs_list[1:],axis=1)
    # decoder input ids 
    # Shape: batch_size*[max_output_sequence_len,1]
    dec_input_ids = tf.unstack(tf.expand_dims(tf.stack(outputs_list[:-1],axis=1),2),axis=0)
    # encoder input ids 
    # Shape: batch_size*[max_input_sequence_len+1,1]
    enc_input_ids = [tf.expand_dims(tf.range(2,self.vocab_size),1)]*self.batch_size
    # Look up encoder and decoder inputs
    encoder_inputs = []
    decoder_inputs = []
    #print("I am here 2")
    for i in range(self.batch_size):
      encoder_inputs.append(tf.gather_nd(embedding_table_list[i], enc_input_ids[i]))
      decoder_inputs.append(tf.gather_nd(embedding_table_list[i], dec_input_ids[i]))
    # Shape: [batch_size,max_input_sequence_len+1,2]
    encoder_inputs = tf.stack(encoder_inputs,axis=0)
    # Shape: [batch_size,max_output_sequence_len,2]
    decoder_inputs = tf.stack(decoder_inputs,axis=0)
    # Stack encoder cells if needed
    if num_layers > 1:
      fw_enc_cell = tf.contrib.rnn.MultiRNNCell([cell(rnn_size) for _ in range(num_layers)]) 
      bw_enc_cell = tf.contrib.rnn.MultiRNNCell([cell(rnn_size) for _ in range(num_layers)]) 
    else:
      fw_enc_cell = cell(rnn_size)
      bw_enc_cell = cell(rnn_size)    
    # Tile inputs if forward only
    if self.forward_only:
      # Tile encoder_inputs and enc_input_lens
      encoder_inputs = tf.contrib.seq2seq.tile_batch(encoder_inputs,beam_width)
      enc_input_lens = tf.contrib.seq2seq.tile_batch(enc_input_lens,beam_width)
    
    # Encode input to obtain memory for later queries
    memory,_ = tf.nn.bidirectional_dynamic_rnn(fw_enc_cell, bw_enc_cell, encoder_inputs, enc_input_lens, dtype=tf.float32)
    # Shape: [batch_size(*beam_width), max_input_sequence_len+1, 2*rnn_size]
    memory = tf.concat(memory, 2) 
    # PointerWrapper
    pointer_cell = PointerWrapper(cell(rnn_size), attention_size, memory)
    
    # Stack decoder cells if needed
    if num_layers > 1:
      dec_cell = tf.contrib.rnn.MultiRNNCell([cell(rnn_size) for _ in range(num_layers-1)]+[pointer_cell])
    else:
      dec_cell = pointer_cell
    
    
    # Different decoding scenario
    if self.forward_only:
      # Tile embedding_table
      tile_embedding_table = tf.tile(tf.expand_dims(embedding_table,1),[1,beam_width,1,1])
      # Customize embedding_lookup_fn
      def embedding_lookup(ids):
        # Note the output value of the decoder only ranges 0 to max_input_sequence_len
        # while embedding_table contains two more tokens' values 
        # To get around this, shift ids
        # Shape: [batch_size,beam_width] 
        ids = ids+2
        # Shape: [batch_size,beam_width,vocab_size]
        one_hot_ids = tf.cast(tf.one_hot(ids,self.vocab_size), dtype=tf.float32)
        # Shape: [batch_size,beam_width,vocab_size,1]
        one_hot_ids = tf.expand_dims(one_hot_ids,-1)
        # Shape: [batch_size,beam_width,features_size]
        next_inputs = tf.reduce_sum(one_hot_ids*tile_embedding_table, axis=2)
        return next_inputs
      # Do a little trick so that we can use 'BeamSearchDecoder'
      shifted_START_ID = START_ID - 2
      shifted_END_ID = END_ID - 2
      
      # Do a little trick so that we can use 'BeamSearchDecoder'
      shifted_START_ID = START_ID - 2
      shifted_END_ID = END_ID - 2
      # Beam Search Decoder
      decoder = tf.contrib.seq2seq.BeamSearchDecoder(dec_cell, embedding_lookup, 
                                          tf.tile([shifted_START_ID],[self.batch_size]), shifted_END_ID, 
                                          dec_cell.zero_state(self.batch_size*beam_width,tf.float32), beam_width)
      # Decode
      outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
      # predicted_ids
      # Shape: [batch_size, max_output_sequence_len,  beam_width]
      predicted_ids = outputs.predicted_ids
      # Transpose predicted_ids
      # Shape: [batch_size, beam_width, max_output_sequence_len]
      self.predicted_ids = tf.transpose(predicted_ids,[0,2,1])
    else:
      #print("sssssssssdecodring")
      # Get the maximum sequence length in current batch
      cur_batch_max_len = tf.reduce_max(dec_input_lens)
      # Training Helper
      helper = tf.contrib.seq2seq.TrainingHelper(decoder_inputs, dec_input_lens)    
      # Basic Decoder
      decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, dec_cell.zero_state(self.batch_size,tf.float32)) 
      # Decode
      outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True)
      # logits
      logits = outputs.rnn_output
      print("logits in mishavad",logits)
      # predicted_ids_with_logits
      self.predicted_ids_with_logits=tf.nn.top_k(logits)
      # Pad logits to the same shape as targets
      logits = tf.concat([logits,tf.ones([self.batch_size,self.max_output_sequence_len-cur_batch_max_len,self.max_input_sequence_len+1])],axis=1)
      # Subtract target values by 2
      # because prediction output ranges from 0 to max_input_sequence_len+1
      # while target values are from 0 to max_input_sequence_len + 3 
      self.shifted_targets = (self.targets - 2)*self.dec_input_weights
      # Losses
      losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.shifted_targets, logits=logits)
      # Total loss
      self.loss = tf.reduce_sum(losses*tf.cast(self.dec_input_weights,tf.float32))/self.batch_size
      # Get all trainable variables
      parameters = tf.trainable_variables()
      # Calculate gradients
      gradients = tf.gradients(self.loss, parameters)
      # Clip gradients
      clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
      # Optimization
      #optimizer = tf.train.GradientDescentOptimizer(self.init_learning_rate)
      optimizer = tf.train.AdamOptimizer(self.init_learning_rate)
      # Update operator
      self.update = optimizer.apply_gradients(zip(clipped_gradients, parameters),global_step=self.global_step)
      # Summarize
      tf.summary.scalar('loss',self.loss)
      for p in parameters:
        tf.summary.histogram(p.op.name,p)
      for p in gradients:
        tf.summary.histogram(p.op.name,p)
      # Summarize operator
      self.summary_op = tf.summary.merge_all()
      #DEBUG PART
      self.debug_var = logits
      #/DEBUG PART

    # Saver
    self.saver = tf.train.Saver(tf.global_variables())

  def step(self, session, inputs, enc_input_weights, outputs, dec_input_weights,X,Y):
    '''Run a step of the model feeding the given inputs.
    Args:
      session: tensorflow session to use.
      inputs: the point positions in 2D coordinate. shape: [batch_size,max_input_sequence_len,2]
      enc_input_weights: the weights of encoder input points. shape: [batch_size,max_input_sequence_len]              
      outputs: the point indexes in inputs. shape: [batch_size,max_output_sequence_len+1] 
      dec_input_weights: the weights of decoder input points. shape: [batch_size,max_output_sequence_len] 
    '''
    #Fill up inputs 
    input_feed = {}
    input_feed[self.inputs] = inputs
    input_feed[self.enc_input_weights] = enc_input_weights
    if self.forward_only==False:
      input_feed[self.outputs] = outputs
      input_feed[self.dec_input_weights] = dec_input_weights

    #Fill up outputs
    if self.forward_only:
      output_feed = [self.predicted_ids]
    else:
      output_feed = [self.update, self.summary_op, self.loss, self.predicted_ids_with_logits, self.shifted_targets, self.debug_var]

    #Run step
    outputs=train(X, Y, modal, 128, 20)


    #Return
    if self.forward_only:
      return outputs[0]
    else:
      return outputs[1],outputs[2],outputs[3],outputs[4],outputs[5]

## Train Segment model
* "model sequence generator" generates sequences of sentences of size pointer_input_size and return corrosponding Y and embeddings of them. Also initialize enc_input_weights and dec_input_weights based on max_input_seq and max_output_seq
* "get batch" makes batches of sequences to fit in the pointer network.
* "build model" builds a pointer network and assign all parameters based on how the sequences are generated.
* "fit data" fits data to pointer_network and run it step by step.


In [161]:
import tensorflow as tf
import numpy as np 
import time
import os
import numpy


class Pointer(object):
 def __init__(self, forward_only, X, Y,seq_size,context_size):
    self.forward_only = forward_only
    self.graph = tf.Graph()
    self.X=X
    self.Y=Y
    self.seq_size=seq_size
    self.context_size=context_size
    with self.graph.as_default():
      self.sess = tf.Session()
      self.sess.run(tf.global_variables_initializer()) 
      #self.optimize()
      #init_op = tf.initialize_all_variables()
      #self.sess.run(init_op)
      
      
    
    
    self.build_model()
    self.pointer_sequence_generator(X,Y,seq_size,context_size)
    
 def pointer_sequence_generator(self,X_train,  Y_train, POINTER_INPUT_SIZE,ONE_SIDE_CONTEXT_SIZE):  
        inputs = []
        enc_input_weights = []
        outputs = []
        dec_input_weights = []
        for Xi_train, Yi_train in zip(X_train, Y_train):
               
                Xi_train, Yi_train = Xi_train.reshape((1,) + Xi_train.shape), Yi_train.reshape((1,) + Yi_train.shape)   # Convert to format of 1 document
                for batch_X_left, batch_X_mid, batch_X_right, batch_Y_mid in batch_gen_consecutive_context_segments_from_big_seq("train",Xi_train, Yi_train, 10,10):
                    
                    embeddings = new_model.predict_on_batch([batch_X_left, batch_X_mid, batch_X_right]) 
                    enc_input = []
                    enc_input_len = embeddings.shape[0]
                    enc_input += [0]*((10-enc_input_len)*2700) 
                    enc_input = np.array(enc_input).reshape([-1,2700])
                    enc_input=numpy.concatenate((enc_input,embeddings)) #pad to 10\
                    inputs.append(enc_input)
                    weight = np.zeros(10)
                    weight[:enc_input_len]=1
                    enc_input_weights.append(weight)
                    index=[]
                    count1=0
                    for i in range(len(batch_Y_mid)):
                     if batch_Y_mid[i]==[1]:
                        index.append(i+1)
                        count1=count1+1
                    output=[0]
                    if count1>0:
                        
                     for i in index:
                      # Add 2 to value due to the sepcial tokens
                       output.append(int(i)+2)
                    else:
                        output.append(0)
                    output.append(2)
                    dec_input_len = len(output)-1
                    output += [0]*(5-dec_input_len)
                    output = np.array(output)
                    outputs.append(output)
                    weight = np.zeros(5)
                    weight[:dec_input_len]=1
                    dec_input_weights.append(weight)
        
        self.inputs = np.stack(inputs)
        self.enc_input_weights = np.stack(enc_input_weights)
        self.outputs = np.stack(outputs)
        self.dec_input_weights = np.stack(dec_input_weights)

        print("Load inputs:            " +str(self.inputs.shape))
        print("Load enc_input_weights: " +str(self.enc_input_weights.shape))
        print("Load outputs:           " +str(self.outputs.shape))
        print("Load dec_input_weights: " +str(self.dec_input_weights.shape))
 def get_batch(self):
    data_size = self.inputs.shape[0]
    sample = np.random.choice(data_size,128,replace=True)
    return self.inputs[sample],self.enc_input_weights[sample],\
      self.outputs[sample], self.dec_input_weights[sample]
 def build_model(self):
    with self.graph.as_default():
      # Build model
      self.model = PointerNet(batch_size=128, 
                    max_input_sequence_len=10, 
                    max_output_sequence_len=5, 
                    rnn_size=128, 
                    attention_size=128, 
                    num_layers=1,
                    beam_width=2, 
                    learning_rate=0.001, 
                    max_gradient_norm=5.0, 
                    forward_only=False)
      # Prepare Summary writer
      self.writer = tf.summary.FileWriter('log' + '/train',self.sess.graph)
      print("Created model with fresh parameters.")
      # Try to get checkpoint
      #ckpt = tf.train.get_checkpoint_state('log')
      #if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        #print("Load model parameters from %s" % ckpt.model_checkpoint_path)
        #self.model.saver.restore(self.sess, ckpt.model_checkpoint_path)
      #else:
       #print("Created model with fresh parameters.")
        #self.sess.run(tf.global_variables_initializer())
 def fit(self):
    
    inputs,enc_input_weights, outputs, dec_input_weights = \
                  self.get_batch()
      #print("inputs",inputs[0])
    summary, step_loss, predicted_ids_with_logits, targets, debug_var = \
                  self.model.step(self.sess, inputs, enc_input_weights, outputs, dec_input_weights,self.X,self.Y)
    

 def run(self):
      self.fit()



## Train Segment Model with multi_layer pointer network

In [159]:
POINTER_INPUT_SIZE=10
ONE_SIDE_CONTEXT_SIZE = 5
epochs=2
weight_size = 256
batch_size = 250
SAMPLE_TYPE_wiki, X_wiki, Y_wiki, trained_sample_handler = get_input(sample_type=7, shuffle_documents=True, pad=False)

X_train, Y_train, X_test, Y_test = split_data(X_wiki, Y_wiki, train_split=0.8)
pointer = Pointer(False, X_train, Y_train,POINTER_INPUT_SIZE,ONE_SIDE_CONTEXT_SIZE)


Total WIKIPEDIA(test) data: 300 chapters, 2265 paragraphs and 58578 sentences
Samples Loading took 0.912585973739624 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 95.11669611930847
Converting doc to martices 51 and the time passed is: 126.48734998703003
Converting doc to martices 101 and the time passed is: 159.8767910003662
Converting doc to martices 151 and the time passed is: 195.5670657157898
Converting doc to martices 201 and the time passed is: 225.92934703826904
Converting doc to martices 251 and the time passed is: 256.83310198783875
Total samples: 300
SHUFFLE: Shuffled input document order! (X: (300,) , Y: (300,) )
X_train: (241,)  | Y_train: (241,)
X_test: (59,)  | Y_test: (59,)
<tf.Variable 'special_token_embedding:0' shape=(3, 2700) dtype=float32_ref>
logits in mishavad Tensor("decoder/transpose:0", shape=(128, ?, 11), dtype=float32)
Created model with fresh parameters.
Load inputs:            (4611, 10, 2700)
Load enc_inp

In [174]:
SAMPLE_TYPE_bio, X_bio, Y_bio, trained_sample_handler = get_input(sample_type=5, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
SAMPLE_TYPE_cli, X_cli, Y_cli, trained_sample_handler = get_input(sample_type=4, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
SAMPLE_TYPE_fic, X_fic, Y_fic, trained_sample_handler = get_input(sample_type=6, shuffle_documents=False, pad=False, trained_sent2vec_model=trained_sample_handler)
pointer.run()

Total BIOGRAPHY data: 11 chapters, 298 paragraphs and 2285 sentences
Samples Loading took 0.035481929779052734 seconds
Loading word2vec model
Model Loaded!
Converting doc to martices 1 and the time passed is: 115.69974994659424
Total samples: 1
Total CLINICAL data: 227 chapters, 909 paragraphs and 31868 sentences
Samples Loading took 1.502230167388916 seconds
Converting doc to martices 1 and the time passed is: 1.5024871826171875
Converting doc to martices 51 and the time passed is: 54.07913088798523
Converting doc to martices 101 and the time passed is: 96.84037804603577
Converting doc to martices 151 and the time passed is: 130.307532787323
Converting doc to martices 201 and the time passed is: 151.52893495559692
Total samples: 227
 Loading FICTION: testing data 
Total FICTION data: 82 chapters, 1245 paragraphs and 27551 sentences
Samples Loading took 1.5039491653442383 seconds
Converting doc to martices 1 and the time passed is: 1.5042529106140137
Converting doc to martices 51 and t

, Epoc 1/20, BatchSz: 128 sents, Batch: 111/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 0.5576, Acc: 0.9652, Rec: 0.9652351
, Epoc 1/20, BatchSz: 128 sents, Batch: 112/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:02 || Loss: 0.6750, Acc: 0.9579, Rec: 0.9579351
, Epoc 1/20, BatchSz: 128 sents, Batch: 113/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 0.7286, Acc: 0.9545, Rec: 0.9545351
, Epoc 1/20, BatchSz: 128 sents, Batch: 114/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 0.3757, Acc: 0.9766, Rec: 0.9766351
, Epoc 1/20, BatchSz: 128 sents, Batch: 115/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 0.3757, Acc: 0.9766, Rec: 0.9766351
, Epoc 1/20, BatchSz: 128 sents, Batch: 116/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 0.5010, Acc: 0.9688, Rec: 0.9688351
, Epoc 1/20, BatchSz: 128 sents, Batch: 117/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:01 || Loss: 2.0684, Acc: 0.8710, Rec: 0.8710351
, Epoc 1/20, BatchSz: 128 sents, B

, Epoc 1/20, BatchSz: 128 sents, Batch: 229/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.4858, Acc: 0.9697, Rec: 0.9697351
, Epoc 1/20, BatchSz: 128 sents, Batch: 230/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.7633, Acc: 0.9524, Rec: 0.9524351
, Epoc 1/20, BatchSz: 128 sents, Batch: 231/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.7515, Acc: 0.9531, Rec: 0.9531351
, Epoc 1/20, BatchSz: 128 sents, Batch: 232/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.4453, Acc: 0.9722, Rec: 0.9722351
, Epoc 1/20, BatchSz: 128 sents, Batch: 233/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.6891, Acc: 0.9570, Rec: 0.9531351
, Epoc 1/20, BatchSz: 128 sents, Batch: 234/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 2.6717, Acc: 0.8333, Rec: 0.8333351
, Epoc 1/20, BatchSz: 128 sents, Batch: 235/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.8144, Acc: 0.9492, Rec: 0.9453351
, Epoc 1/20, BatchSz: 128 sents, B

, Epoc 1/20, BatchSz: 128 sents, Batch: 347/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.0000, Acc: 1.0000, Rec: 1.0000351
, Epoc 1/20, BatchSz: 128 sents, Batch: 348/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.0000, Acc: 1.0000, Rec: 1.0000351
, Epoc 1/20, BatchSz: 128 sents, Batch: 349/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.7516, Acc: 0.9531, Rec: 0.9531351
, Epoc 1/20, BatchSz: 128 sents, Batch: 350/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.3490, Acc: 0.9783, Rec: 0.9783351
, Epoc 1/20, BatchSz: 128 sents, Batch: 351/351, Speed: 0.00 secs/batch, TimeRemain: 0:00:00 || Loss: 0.6286, Acc: 0.9608, Rec: 0.9608351
, Epoc 1/20, BatchSz: 128 sents, Batch: 352/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.7514, Acc: 0.9531, Rec: 0.9531351
, Epoc 1/20, BatchSz: 128 sents, Batch: 353/351, Speed: 0.01 secs/batch, TimeRemain: 0:00:00 || Loss: 0.0000, Acc: 1.0000, Rec: 1.0000351
, Epoc 1/20, BatchSz: 128 sents, B

>> Summary (Fiction):
AVG segment length in test data: 24.157979, std: 18.248431
WinDiff metric (Windiff_r):: avg: 0.306499 | std: 0.129458 | min: 0.000000 | max: 0.447837
WinDiff metric (Windiff_e):: avg: 0.305226 | std: 0.129384 | min: 0.000000 | max: 0.446701
Pk metric:: avg: 0.377373 | std: 0.080859 | min: 0.000000 | max: 0.475827
___________________________________
